--------------

# Modules

In [ ]:
import pandas as pd

from sklearn.preprocessing import StandardScaler

import scipy.stats as stats
import numpy as np

import os

from tools.class_plots import Plots
from tools.class_utils import Utils
from tools.class_entropy import Entropy
from tools.class_coherence import Coherence
from tools.class_graph import Graph
from tools.class_time import Time

In [ ]:
# Plots.plot_one_group(df_tgt_pre_ctrl, channels, params = [4, 5], title = "[TARGET] CTRL", l = ["CTRL"], c = ["b"], size = (17, 10))

# Plots.plot_two_groups(df_tgt_pre_ctrl, df_tgt_post_ctrl, best_channels, [2, 5], title = "[TARGET] PRE-POST: CTRL", l = ["PRE", "POST"], c = ["g","brown"], size = (18, 7))

# Plots.plot_three_groups(df_tgt_pre_ctrl, df_tgt_pre_plcb, df_tgt_pre_exp, best_channels, [2, 5], title = "[TARGET] PRE: CTRL, PLCB, and EXP", l = ["CTRL", "PLCB", "EXP"], c = ["b","black","r"], size = (18, 7))

# Plots.plot_components(df_tgt_pre_ctrl, df_tgt_pre_plcb, df_tgt_pre_exp, "Fz", title = '[Target] - PRE: CTRL, PLCB and EXP', l = ["CTRL","PLCB","EXP"], size = (10,5))

# Plots.plot_band_three_groups(df_tgt_pre_ctrl, df_tgt_pre_plcb, df_tgt_pre_exp, best_channels, band = "delta", params = [2,5], title = "[Target - Delta] PRE: CTRL, PLCB and EXP", l = ["CTRL", "PLCB", "EXP"], c = ["b","black","r"])

# Plots.plot_band_two_groups(df_tgt_pre_ctrl, df_tgt_post_ctrl, best_channels, band = "delta", params = [2, 5], title = "[Target - Delta] CTRL: PRE-POST", l = ["PRE", "POST"], c = ["g","brown"], size = (18, 7))

# Plots.plot_groups_by_features(df_feat_tgt_post, best_channels, group = ["CONTROL", "EXP"], params = [2,5], title = "[Target] POST: CTRL vs EXP", l = ["CTRL", "EXP"], c = ["b", "r"], size = (18, 10), b_id = False)

def get_matrix(df, grupo, comp, ch, feat):
    return df[(df["grupo"]==grupo)&(df["comp"]==comp)&(df["ch"]==ch)][feat]

def get_id(df, grupo, comp, ch):
    return df[(df["grupo"]==grupo)&(df["comp"]==comp)&(df["ch"]==ch)]["id"]

def normalize(df_feat, components):

    # Lista para almacenar DataFrames normalizados por componente
    dfs_normalizados = []

    # Iterar sobre cada componente único
    for c in components:
        # Filtrar filas del componente actual
        mask = df_feat['comp'] == c
        df_componente = df_feat[mask].copy()
        
        # Normalizar solo las columnas de características (desde la columna 6 en adelante)
        scaler = StandardScaler(with_mean=True, with_std=True)
        X_norm = scaler.fit_transform(df_componente.iloc[:, 6:])
        
        # Crear un nuevo DataFrame con las columnas normalizadas
        df_norm = pd.DataFrame(
            X_norm,
            columns=df_componente.columns[6:],
            index=df_componente.index  # Mantener los índices originales
        )
        
        # Concatenar con las columnas no normalizadas (0 a 5)
        df_final_componente = pd.concat([
            df_componente.iloc[:, :6],  # Metadatos sin normalizar
            df_norm                    # Características normalizadas
        ], axis=1)
        
        dfs_normalizados.append(df_final_componente)

    # Unir todos los DataFrames normalizados en uno solo
    # Ordenar por índice original si es necesario
    df_feat_norm = pd.concat(dfs_normalizados, axis=0).sort_index()

    return df_feat_norm



# Information

## Grupo de psicología

(-) **TAREAS ODDBALL**

En el análisis bibliográfico sugieren emplear una ventana de -100ms a 750ms para capturar la respuesta al estímulo. Además, hablan de una componente bastante interesante llamada *novelty P3* (nP300), que aparece cuando el estímulo es completamente nuevo. 

- Sugieren estudiar esta componente.

En personas de avanzada edad, el envejecimiento presenta una influencia en la amplitud y latencia de **P300**. En general, las evidencias apuntan hacia un aumento de la latencia y una disminución de la amplitud.

- Sugieren que una mayor latencia es sinónimo de un peor desempeño. 
- En comparaciones de ancianos y adultos jóvenes, los ancianos muestran amplitudes mayores en regiones frontales y más bajos en regiones posteriores.
- Estos cambios parecen reflejarse en la componente **nP300**.

En varios estudios, los participantes mayores con baja actividad física mostraron valores **AUC** más altos en comparación con los demás grupos, lo que sugiere una mayor asignación de recursos neuronales en respuesta a desafíos cognitivos simples.

(-) **OJOS ABIERTOS Y CERRADOS**

En ancianos con edades comprendidas entre 75 y 84 años aparece un incremento en la actividad de las ondas lentas *delta* y *theta*, especialmente en las regiones temporales y occipitales izquierdas, cuando se compara con individuos entre 65 y 74 años. 

- En individuos mayores de 85 años, se observa un desplazamiento hacia frecuencias más bajas, con un pico espectral entre 9.0-10.0 Hz. 
- En un espectrograma se debería observar una franja en frecuencias bajas, como *delta* y *theta*.

Respecto a las bandas de frecuencia en ancianos, la frecuencia pico de la banda alfa (7-14 Hz) disminuye con la edad, lo que sugiere una reducción en la actividad cerebral asociada con estados de relajación y atención. La potencia de la banda beta (15-30 Hz) también disminuye en ancianos con una mayor carga cardiometabólica, lo que puede indicar una menor actividad cognitiva y una menor capacidad de respuesta rápida.

- La frecuencia pico individual de la banda *alpha* (IAF, frecuencia discreta de un individuo con la potencia más alta) se ha asociado positivamente con la resolución de interferencias en tareas de memoria y con un mejor rendimiento en tareas de atención sostenida.

El grupo de psicología remarca la importancia de la **variabilidad individual**. Aunque se puedan observar patrones generales, existe una considerable variabilidad individual en todos los parámetros del EEG, lo que sugiere que los cambios en el EEG con el envejecimiento no son uniformes para todos los individuos.

Existe una propiedad sobre el **comportamiento apreiódico** de la actividad cerebral, que muestra una correlación con el rendimiento cognitivo. En varios estudios, los ancianos mostraron una menor actividad aperiódica global.

(-) **NEXT STEPS**

- Debemos observar el comportamiento de la actividad en ondas lentas *delta* y *theta*.

- Frecuencia Pico de Banda Alfa (IAF): este parámetro disminuye con la edad, indicando menor eficiencia neural. En el grupo experimental, podría esperarse que las frecuencias pico sean más altas en el instante post.

- Frecuencia Pico de Banda Beta: este parámetro disminuye con la edad, indicando menor actividad cognitiva.

- Deberíamos tener en cuenta una ventana de -100ms antes del estímulo *oddball*.
- En el grupo experimental podríamos esperar una menor latencia y mayor amplitud en post, en comparación con los otros dos grupos.
- Convendría analizar el ERP *novelty P3* (nP300).


## Preprocessing steps

En la estimulación *oddball* se han recopilado dos mediciones EEG con el nombre *oddball Nº1* y *oddball Nº2*. Por lo tanto, cada sujeto debería poseer dos archivos (.edf) y (.mat), que guardan los timings y tipo de estímulo, respectivamente. Sin embargo, algunos de ellos solo poseen información sobre una medición. 
En cuanto al número de estímulos, sumando los *target* y *estándar*, deberían tener un total de 100, pero los rangos varían entre [80-102]. Los sujetos que tienen alguna anomalía en la duración de la medición y con el número o tipo de estímulos están controlados, anotados y separados del análisis.

Para este primer apartado, hemos seleccionado todos las mediciones que poseen Nºestímulos > 90 con su respectiva información del tipo y *timing*. Así, podemos trabajar con la mayoría de las grabaciones descartando únicamente 7 por los motivos que aparecen, a continuación:

- POST_Exp_33_PB_ODDBALL_1: su archivo (.edf) no posee ningún *timing* de los estímulos.
- POST_Exp_31_PS_ODDBALL_1: su archivo (.edf) no posee ningún *timing* de los estímulos.
- POST_Exp_47_LO_ODDBALL_1: su archivo (.edf) posee 86 estímulos marcados.
- PRE_Exp_34_AB_ODDBALL_1: su archivo (.edf) posee 80 estímulos marcados. 
- PRE_Exp_34_AB_ODDBALL_2: su archivo (.edf) posee 80 estímulos marcados. 
- PRE_Exp_38_FL_ODDBALL_2: su archivo (.edf) posee 80 estímulos marcados. 
- PRE_Exp_65_MSR_ODDBALL_2: su archivo (.edf) posee 12 estímulos marcados.

Ahora, en cada grabación EEG y por cada canal, detectamos el instante inicial de los estímulos *target* y *standard* por separado. Sabemos que existe información interesante unos instantes antes del disparo, así que hemos decidido seleccionar su comienzo 40 ms antes de lo que marca el *timing*. El tamaño de ventana seleccionado es de [-200, +700] ms cubriendo una línea base y todas las componentes de respuesta al estímulo.

$$
\begin{array}{|c|c|}
\hline
\textbf{Ventana} & \textbf{[ms]} \\
\hline
\text{$t_{o}$} & -200 \\ \hline 
\text{$t_{f}$} & +700 \\ \hline
\end{array}
$$

En cada registro EEG y por cada uno de sus 20 canales, se selecciona una ventana mediana de [-200, +700] ms para respuesta *target* y otra para respuesta *standard*. Así, tendremos 20 señales *target* y *standard* por sujeto/regristro, lo que nos permitirá extraer las componentes ERP que aparecen, a continuación: 

$$Event\ Related\ Potentials\ (ERP)\ Components$$

$$
\begin{array}{|c|c|c|}
\hline
\textbf{Componentes} & \textbf{$t_{o}$ [ms]} & \textbf{$t_{f}$ [ms]} \\
\hline
\text{baseline} & -200 & -100 \\ \hline
\text{pre-trigger} & -100 & 0 \\ \hline
\text{post-trigger} & 0 & 80 \\ \hline
\text{P1} & 80 & 130 \\ \hline
\text{N1} & 130 & 200 \\ \hline
\text{P2} & 200 & 300 \\ \hline
\text{N2} & 300 & 360 \\ \hline
\text{P3} & 360 & 600 \\ \hline
\end{array}
$$

## *Data Distribution*

$$
\begin{array}{|c|c|c|c|c|c|c|c|c|c|}
\hline
\textbf{$n_{test}$} & \textbf{instante}  & \textbf{grupo} & \textbf{id} & \textbf{type} & \textbf{comp} & \textbf{P8} & \textbf{T8} & \textbf{...} & \textbf{P7} \\
\hline
\text{1} & \text{PRE} & \text{CONTROL} & \text{1} & \text{tgt} & \text{all} & \text{...} & \text{...} & \text{...} & \text{...}\\
\hline
\text{1} & \text{PRE} & \text{CONTROL} & \text{1} & \text{tgt} & \text{all} & \text{...} & \text{...} & \text{...} & \text{...}\\
\hline
\text{1} & \text{PRE} & \text{CONTROL} & \text{1} & \text{tgt} & \text{all} & \text{...} & \text{...} & \text{...} & \text{...}\\
\hline
\text{...} & \text{...} & \text{...} & \text{...} & \text{...} & \text{...} & \text{...} & \text{...} & \text{...} & \text{...}\\
\hline
\text{2} & \text{POST} & \text{EXP} & \text{25} & \text{std} & \text{P1} & \text{...} & \text{...} & \text{...} & \text{...}\\
\hline
\text{2} & \text{POST} & \text{EXP} & \text{25} & \text{std} & \text{P1} & \text{...} & \text{...} & \text{...} & \text{...}\\
\hline
\text{2} & \text{POST} & \text{EXP} & \text{25} & \text{std} & \text{P1} & \text{...} & \text{...} & \text{...} & \text{...}\\
\hline
\text{...} & \text{...} & \text{...} & \text{...} & \text{...} & \text{...} & \text{...} & \text{...} & \text{...} & \text{...}\\
\hline
\text{1} & \text{SEG} & \text{CONTROL} & \text{87} & \text{tgt} & \text{P3} & \text{...} & \text{...} & \text{...} & \text{...}\\
\hline
\text{1} & \text{SEG} & \text{CONTROL} & \text{87} & \text{tgt} & \text{P3} & \text{...} & \text{...} & \text{...} & \text{...}\\
\hline
\text{1} & \text{SEG} & \text{CONTROL} & \text{87} & \text{tgt} & \text{P3} & \text{...} & \text{...} & \text{...} & \text{...}\\
\hline
\end{array}
$$

# Visualize stimuli

La siguiente agrupación se basa en el Sistema Internacional 10-20 para la colocación de electrodos EEG, que es el estándar en neurociencia para referenciar ubicaciones en el cuero cabelludo.

Las letras indican el lóbulo donde se ubica el electrodo:

$$
\begin{array}{|c|c|}
\hline
\textbf{Abreviatura} & \textbf{Lóbulo} \\
\hline
\text{Fp} & \text{Frontopolar} \\
\text{F} & \text{Frontal} \\
\text{C} & \text{Central} \\
\text{P} & \text{Parietal} \\
\text{O} & \text{Occipital} \\
\text{T} & \text{Temporal} \\
\hline
\end{array}
$$

Los números indican si el electrodo está en el lado izquierdo (números impares), derecho (números pares) o en la línea media (letras terminadas en "z").

In [ ]:
# Paths with files (.csv)
path_tgt = "E:/TFM/CLUSTERING_ALL_ICA_by_segments/ODDBALL/stimulus_tgt.csv"
path_std = "E:/TFM/CLUSTERING_ALL_ICA_by_segments/ODDBALL/stimulus_std.csv"

# Load DataFrame for every type of stimulus
df_w_tgt = pd.read_csv(path_tgt, header=0, delimiter=';')
df_w_std = pd.read_csv(path_std, header=0, delimiter=';')

# Common information about both DataFrames, where: df_std.columns == df_tgt.columns
channels = df_w_std.columns[6:].to_list()
info = df_w_std.columns[:6].to_list()
components = df_w_std["comp"].unique()

# Seleccionamos instantes y grupos de TARGET DataFrames
df_tgt_ctrl = df_w_tgt[df_w_tgt["grupo"] == "CONTROL"]# select CTRL subjects
df_tgt_plcb = df_w_tgt[df_w_tgt["grupo"] == "PLCB"]# select PLCB subjects
df_tgt_exp = df_w_tgt[df_w_tgt["grupo"] == "EXP"]# select EXP subjects

df_tgt_pre_ctrl = df_w_tgt[(df_w_tgt["instante"]=="PRE")&(df_w_tgt["grupo"]=="CONTROL")]
df_tgt_pre_plcb = df_w_tgt[(df_w_tgt["instante"]=="PRE")&(df_w_tgt["grupo"]=="PLCB")]
df_tgt_pre_exp = df_w_tgt[(df_w_tgt["instante"]=="PRE")&(df_w_tgt["grupo"]=="EXP")]

df_tgt_post_ctrl = df_w_tgt[(df_w_tgt["instante"]=="POST")&(df_w_tgt["grupo"]=="CONTROL")]
df_tgt_post_plcb = df_w_tgt[(df_w_tgt["instante"]=="POST")&(df_w_tgt["grupo"]=="PLCB")]
df_tgt_post_exp = df_w_tgt[(df_w_tgt["instante"]=="POST")&(df_w_tgt["grupo"]=="EXP")]

df_tgt_seg_ctrl = df_w_tgt[(df_w_tgt["instante"]=="SEGUIMIENTO")&(df_w_tgt["grupo"]=="CONTROL")]
df_tgt_seg_plcb = df_w_tgt[(df_w_tgt["instante"]=="SEGUIMIENTO")&(df_w_tgt["grupo"]=="PLCB")]
df_tgt_seg_exp = df_w_tgt[(df_w_tgt["instante"]=="SEGUIMIENTO")&(df_w_tgt["grupo"]=="EXP")]

# Seleccionamos instantes y grupos de STANDARD DataFrames
df_std_ctrl = df_w_std[df_w_std["grupo"] == "CONTROL"]# select CTRL subjects
df_std_plcb = df_w_std[df_w_std["grupo"] == "PLCB"]# select PLCB subjects
df_std_exp = df_w_std[df_w_std["grupo"] == "EXP"]# select EXP subjects

df_std_pre_ctrl = df_w_std[(df_w_std["instante"]=="PRE")&(df_w_std["grupo"]=="CONTROL")]
df_std_pre_plcb = df_w_std[(df_w_std["instante"]=="PRE")&(df_w_std["grupo"]=="PLCB")]
df_std_pre_exp = df_w_std[(df_w_std["instante"]=="PRE")&(df_w_std["grupo"]=="EXP")]

df_std_post_ctrl = df_w_std[(df_w_std["instante"]=="POST")&(df_w_std["grupo"]=="CONTROL")]
df_std_post_plcb = df_w_std[(df_w_std["instante"]=="POST")&(df_w_std["grupo"]=="PLCB")]
df_std_post_exp = df_w_std[(df_w_std["instante"]=="POST")&(df_w_std["grupo"]=="EXP")]

df_std_seg_ctrl = df_w_std[(df_w_std["instante"]=="SEGUIMIENTO")&(df_w_std["grupo"]=="CONTROL")]
df_std_seg_plcb = df_w_std[(df_w_std["instante"]=="SEGUIMIENTO")&(df_w_std["grupo"]=="PLCB")]
df_std_seg_exp = df_w_std[(df_w_std["instante"]=="SEGUIMIENTO")&(df_w_std["grupo"]=="EXP")]

# Seleccionamos sujetos comunes entre instantes
df_tgt_pre_ctrl,df_tgt_post_ctrl = Utils.select_common_ids(df_tgt_pre_ctrl, df_tgt_post_ctrl)
df_tgt_pre_plcb,df_tgt_post_plcb = Utils.select_common_ids(df_tgt_pre_plcb, df_tgt_post_plcb)
df_tgt_pre_exp,df_tgt_post_exp = Utils.select_common_ids(df_tgt_pre_exp, df_tgt_post_exp)

eeg_regions = {
    "Frontal": ["Fp1", "Fp2", "F3", "Fz", "F4", "F7", "F8"],
    "Central": ["C3", "Cz", "C4"],
    "Parietal": ["P3", "Pz", "P4"],
    "Occipital": ["O1", "Oz", "O2"],
    "Temporal": ["T7", "T8"],
    "Parieto-temporal": ["P7", "P8"],
    "Parieto-occipital": ["P3", "P4"],
    "Linea-media":["Pz","Cz","Oz"]
}

electrodes = {
    'Fp1': (-0.4, 0.8), 'Fp2': (0.4, 0.8), 
    'F3': (-0.5, 0.4), 'F4': (0.5, 0.4),
    'C3': (-0.6, 0), 'C4': (0.6, 0),
    'F7': (-0.8, 0.6), 'F8': (0.8, 0.6),
    'T7':(-1.0, 0), 'T8': (1.0, 0),
    'Fz': (0, 0.3), 'Cz': (0, 0), 'Pz':(0, -0.3),
    'P7': (-0.8, -0.5),'P3': (-0.5, -0.4), 'P4': (0.5, -0.4), 'P8': (0.8, -0.5),
    'O1': (-0.4, -0.8),'Oz':(0,-0.9), 'O2': (0.4, -0.8)
}

print("Header with info:",info)
print("Channels:",channels)
print("Components:",components)

## Full-Band window

**Objetivo**: 
- Visualizar los estímulos de todos los individuos de manera general, teniendo en cuenta que algunos canales son más informativos que otros.

**Problema**: 
- Contamos con muchos sujetos, algunos con dos pruebas oddball. Graficarlos de forma individual no es viable, ya que requeriría demasiado tiempo.

**Solución**:
- Seleccionar y promediar las ventanas de [-200, 700] ms de todos los CONTROLES. Luego, graficar estas ventanas promediadas por canal para identificar los más informativos.

**Resultado**: 
- Los canales ['P8','T8', 'P4', 'O1', 'Oz', 'O2', 'Pz', 'P3', 'T7', 'P7'] no reflejan de forma nítida la respuesta al estímulo. Vamos a separarlos para una mejor visualización. 

In [ ]:
# Plots.plot_one_group(df_tgt_pre_ctrl, channels, params = [4, 5], title = "[TARGET] CTRL", l = ["CTRL"], c = ["b"], size = (17, 10))
# Plots.plot_one_group(df_std_pre_ctrl, channels, params = [4, 5], title = "[STANDARD] CTRL", l = ["CTRL"], c = ["b"], size = (17, 10))

In [ ]:
best_channels = list(set(channels) ^ set(['P8','T8', 'P4', 'O1', 'Oz', 'O2', 'Pz', 'P3', 'T7', 'P7']))  # Operador XOR (^) en conjuntos

### **Raw**

En nuestra reunión discutimos que las diferencias observadas en términos de amplitud, tanto en instante PRE como POST, podrían ser un efecto enmascarado de posibles outliers. Por ello, se ha realizado el *plot*, que aparece en la próxima sección de código, donde reflejamos las ventanas promedio de todos los sujetos. Ahí, podemos ver que existen ciertos individuos que tuvieron algún tipo de incidencia en la grabación de la prueba, ya que presentan amplitudes abruptas sin sentido. Desgraciadamente, todos los resultados obtenidos e interpretados hasta la fecha son incorrectos porque eran fruto de artefactos.

In [ ]:
# Ruta de la carpeta
path = './raw_data/'

# Load Raw DataFrames
raw_tgt_pre_ctrl = pd.read_csv('./raw_data/tgt_pre_ctrl.csv', header=0, delimiter=';')
raw_tgt_post_ctrl = pd.read_csv('./raw_data/tgt_post_ctrl.csv', header=0, delimiter=';')

raw_tgt_pre_plcb = pd.read_csv('./raw_data/tgt_pre_plcb.csv', header=0, delimiter=';')
raw_tgt_post_plcb = pd.read_csv('./raw_data/tgt_post_plcb.csv', header=0, delimiter=';')

raw_tgt_pre_exp = pd.read_csv('./raw_data/tgt_pre_exp.csv', header=0, delimiter=';')
raw_tgt_post_exp = pd.read_csv('./raw_data/tgt_post_exp.csv', header=0, delimiter=';')

# Plot all stimuli for CTRL
c = ["blue","lightgray", "darkblue", "darkblue"]
dfs = [raw_tgt_pre_ctrl, raw_tgt_post_ctrl]
Plots.all_stimuli(dfs, 'CTRL', c)

# Plot all stimuli for PLCB
c = ["gray","lightgray", "black", "black"]
dfs = [raw_tgt_pre_plcb, raw_tgt_post_plcb]
Plots.all_stimuli(dfs, 'PLCB', c)

# Plot all stimuli for EXP
c = ["lightcoral","lightgray", "darkred", "darkred"]
dfs = [raw_tgt_pre_exp, raw_tgt_post_exp]
Plots.all_stimuli(dfs, 'EXP', c)

- **Outliers**

Identifico los sujetos que presentan amplitudes bruscas sin sentido.

In [ ]:
print("[CTRL]")
print("ID:",raw_tgt_post_ctrl.loc[raw_tgt_post_ctrl["Fz"]>20]["id"].unique())
print("Nº test:",raw_tgt_post_ctrl.loc[raw_tgt_post_ctrl["Fz"]>20]["n_test"].unique())
print()
print("[PLCB]")
print("ID:",raw_tgt_post_plcb.loc[raw_tgt_post_plcb["Fz"]>20]["id"].unique())
print("Nº test:",raw_tgt_post_plcb.loc[raw_tgt_post_plcb["Fz"]>20]["n_test"].unique())
print()
print("[EXP]")
print("ID:",raw_tgt_post_exp.loc[raw_tgt_post_exp["Fz"]>20]["id"].unique())
print("Nº test:",raw_tgt_post_exp.loc[raw_tgt_post_exp["Fz"]>20]["n_test"].unique())
print("ID:",raw_tgt_post_exp.loc[raw_tgt_post_exp["Fz"]<-20]["id"].unique())
print("Nº test:",raw_tgt_post_exp.loc[raw_tgt_post_exp["Fz"]<-20]["n_test"].unique())

- **ERP components**

Partiendo de los datos anteriores (sin filtrar), seleccionamos el **canal Fz** por reflejar de forma nítida las diferencias tan notorias entre instantes y grupos. Al graficar **media** y **mediana**, podemos ver que definitivamente existe un sesgo por varios individuos, lo que ha resultado en unas interpretaciones erróneas.

In [ ]:
Plots.plot_components(raw_tgt_pre_ctrl, raw_tgt_pre_plcb, raw_tgt_pre_exp, ["Fz"], title = 'Target]  PRE: CTRL, PLCB and EXP', l = ["CTRL","PLCB","EXP"], size = (18,5))
Plots.plot_components(raw_tgt_post_ctrl, raw_tgt_post_plcb, raw_tgt_post_exp, ["Fz"], title = 'Target]  POST: CTRL, PLCB and EXP', l = ["CTRL","PLCB","EXP"], size = (18,5))

### **Clean**

Los individuos que han mostrado esas respuestas tan extremas al estímulo han sido descartados de análisis posteriores. La extracción de las ventanas ha sufrido una ligera modificación, es decir, hemos cambiado la media por la mediana de las ventanas en cada sujeto (como explicamos en la sección "Preprocessing steps"). De esta forma, si cada sujeto tiene 15 estímulos **target** por cada uno de los 20 canales, se selecciona la ventana mediana en cada canal para minimizar valores extremos. Después de seleccionar la **mediana** de los estímulos, tenemos una señal más cercana a la realidad del experimento como se puede apreciar en las siguientes gráficas.

In [ ]:
# Plot all stimuli for CTRL
c = ["blue","lightgray", "darkblue", "darkblue"]
dfs = [df_tgt_pre_ctrl, df_tgt_post_ctrl]
Plots.all_stimuli(dfs, 'CTRL', c)

# Plot all stimuli for PLCB
c = ["gray","lightgray", "black", "black"]
dfs = [df_tgt_pre_plcb, df_tgt_post_plcb]
Plots.all_stimuli(dfs, 'PLCB', c)

# Plot all stimuli for EXP
c = ["lightcoral","lightgray", "darkred", "darkred"]
dfs = [df_tgt_pre_exp, df_tgt_post_exp]
Plots.all_stimuli(dfs, 'EXP', c)

- **ERP components**: Target

En la sección anterior, mencionamos que cada sujeto tiene una ventana mediana por canal. Para obtener una visión general de sus respuestas, se ha graficado la ventana promedio y mediana de todos los sujetos. Esta es la razón por la que aparece una gráfica con el título de **Mean** y **Median**. 

En este punto, las diferencias en el instante PRE y POST no son tan notorias como en el caso anterior. 

In [ ]:
Plots.plot_components(df_tgt_pre_ctrl, df_tgt_pre_plcb, df_tgt_pre_exp, ["Cz"], title = 'Target]  PRE: CTRL, PLCB and EXP', l = ["CTRL","PLCB","EXP"], size = (18,5))
Plots.plot_components(df_tgt_post_ctrl, df_tgt_post_plcb, df_tgt_post_exp, ["Cz"], title = 'Target]  POST: CTRL, PLCB and EXP', l = ["CTRL","PLCB","EXP"], size = (18,5))

- **ERP components**: Standard

In [ ]:
Plots.plot_components(df_std_pre_ctrl, df_std_pre_plcb, df_std_pre_exp, ["Fz"], title = '[Standard] - PRE: CTRL, PLCB and EXP', l = ["CTRL","PLCB","EXP"], size = (18,5))
Plots.plot_components(df_std_post_ctrl, df_std_post_plcb, df_std_post_exp, ["Fz"], title = '[Standard] - POST: CTRL, PLCB and EXP', l = ["CTRL","PLCB","EXP"], size = (18,5))

### (-) Target

En primer lugar, graficamos los grupos **CTRL**, **PLCB** y **EXP** en el instante PRE. Ponemos especial énfasis en la homogeneidad de las respuestas, ya que deberían ser similares por tratarse del instante inicial. Se puede apreciar bastante ruido generalizado a lo largo del tiempo. Las respuestas son similares, aunque el grupo **EXP** posee una actividad ligeramente superior. La idea de que parten de condiciones diferentes sigue pareciendo coherente. Sin embargo, estas diferencias no son tan apreciables como el caso de los outliers.

**Groups contrast**

In [ ]:
Plots.plot_three_groups(df_tgt_pre_ctrl, df_tgt_pre_plcb, df_tgt_pre_exp, best_channels, [2, 5], title = "[TARGET]  PRE: CTRL, PLCB, and EXP", l = ["CTRL", "PLCB", "EXP"], c = ["b","black","r"], size = (18, 7))
Plots.plot_two_groups(df_tgt_post_ctrl, df_tgt_post_exp, best_channels, [2, 5], title = "[TARGET]  POST: CTRL vs EXP", l = ["CTRL", "EXP"], c = ["b","r"], size = (18, 7))
Plots.plot_two_groups(df_tgt_post_plcb, df_tgt_post_exp, best_channels, [2, 5], title = "[TARGET]  POST: PLCB vs EXP", l = ["PLCB", "EXP"], c = ["black","r"], size = (18, 7))

-) **PRE: CTRL-PLCB-EXP**

En el instante inicial PRE, podemos observar que los grupos placebo y control presentan una respuesta al estímulo similar; en cambio, el grupo experimental muestra una actividad ligeramente superior en comparación con los demás. Dado que se trata del instante inicial, en el que todos los grupos parten de las mismas condiciones generales, esta diferencia podría deberse a las características individuales de los participantes. 

No obstante, ¿deberíamos darle mucha importancia?. Asumimos que los grupos son equivalentes en el instante PRE, aunque algunos sujetos presenten diferencias individuales que puedan generar variaciones en la señal.

-) **POST: CTRL-EXP**

En el instante POST entre **CTRL** y **EXP**, aparecen diferencias interesantes en la respuesta al estímulo. El grupo **EXP** presenta la componente N1 con una deflexión más positiva y la componente P2 posee una latencia menor.

-) **POST: PLCB-EXP**

En el instante POST entre **PLCB** y **EXP**, aparecen diferencias notables en la respuesta al estímulo. En el instante siguiente al disparo del estímulo, podemos observar una deflexión negativa en el grupo **PLCB** que no aparece en el grupo **EXP**. Además, las deflexiones positivas y negativas están mucho más marcadas en el grupo **PLCB**. 

-) **Conclusión**

En términos biológicos:
- La amplitud en **P3** refleja la cantidad de recursos neuronales que el cerebro ha destinado al procesamiento consciente del estímulo. El pico nos indica que la percepción del estímulo ha necesitado más recursos para alcanzar corteza cerebral. 

- La amplitud en **P2** suele estar relacionada con un procesamiento temprano de la información atencional, es decir, refleja una respuesta rápida del cerebro ante estímulos relevantes. Un aumento en P2 puede sugerir que el cerebro está destinando más recursos a la detección temprana del estímulo. 

- La amplitud en **N2** refleja la cantidad de recursos que el cerebro está dedicando para detectar diferencias entre estímulos.

En conclusión:

- Parece que el grupo  **CTRL** posee una latencia mayor en las componentes P2 y P3 en comparación con los grupos **PLCB** y **EXP**. Los instantes anteriores al dispario en el grupo **EXP** parecen presentar menor variabilidad, mientras que el grupo **CTRL** refleja algunas fluctuaciones. 

- El grupo **PLCB** refleja una ligera disminución de la actividad en instantes siguientes al disparo. Sus deflexiones positivas y negativas poseen un área bajo la curva considerablemente superior al grupo **CTRL** y **EXP**.

**Instants contrast**

In [ ]:
Plots.plot_two_groups(df_tgt_pre_ctrl, df_tgt_post_ctrl, best_channels, [2, 5], title = "[TARGET] PRE-POST: CTRL", l = ["PRE", "POST"], c = ["g","brown"], size = (18, 7))
Plots.plot_two_groups(df_tgt_pre_plcb, df_tgt_post_plcb, best_channels, [2, 5], title = "[TARGET] PRE-POST: PLCB", l = ["PRE", "POST"], c = ["g","brown"], size = (18, 7))
Plots.plot_two_groups(df_tgt_pre_exp, df_tgt_post_exp, best_channels, [2, 5], title = "[TARGET] PRE-POST: EXP", l = ["PRE", "POST"], c = ["g","brown"], size = (18, 7))

-) **Controles**

Al comparar ambos instantes, podemos observar que hay mucha variabilidad a lo largo de la señal. Las componentes N2 y P3 apenas son identificables. 

-) **Placebos**

Al comparar ambos instantes, podemos observar que hay un aumento de amplitud en las componentes P1 y P2. Además, anterior al disparo existe una deflexión positiva que en el instante PRE no aparecía.

-) **Experimentales**

Al comparar ambos instantes dentro de los experimentales, podemos observar que existe una disminución de la actividad cerebral en las componentes N2 y P3. En el instante POST, la N2 y P3 son más diferenciables, pero con valores menos positivos.

-) **Conclusión**

Estas comparaciones dependen de las características individuales e iniciales de cada sujeto. Las gráficas muestran cierta variabilidad, lo que indica que no todos han mostrado el mismo comportamiento. Las conclusiones se han basado principalmente en las gráficas, pero es necesario aplicar análisis estadísticos para confirmar estos resultados.

### (-) Standard

**Groups contrast**

In [ ]:
Plots.plot_three_groups(df_std_pre_ctrl, df_std_pre_plcb, df_std_pre_exp, best_channels, [2, 5], title = "[STANDARD] PRE: CTRL, PLCB, and EXP", l = ["CTRL", "PLCB", "EXP"], c = ["b","black","r"], size = (18, 7))
Plots.plot_two_groups(df_std_post_ctrl, df_std_post_exp, best_channels, [2, 5], title = "[STANDARD] POST: CTRL vs EXP", l = ["CTRL", "EXP"], c = ["b","r"], size = (18, 7))
Plots.plot_two_groups(df_std_post_plcb, df_std_post_exp, best_channels, [2, 5], title = "[STANDARD] POST: PLCB vs EXP", l = ["PLCB", "EXP"], c = ["black","r"], size = (18, 7))

-) **PRE: CTRL-PLCB-EXP**

En el instante inicial PRE, podemos observar que los 3 grupos presentan una respuesta similar.

-) **POST: CTRL-EXP**

En el instante POST, no se observan diferencias entre los 2 grupos.

-) **POST: PLCB-EXP**

En el instante POST, el grupo **PLCB** posee unas deflexiones más pronunciadas en las componente N1 y P2 en comparación con el grupo **EXP**. 


-) **Conclusión**

La respuesta al estímulo muestra una actividad similar entre los tres grupos.


**Instants contrast**

In [ ]:
Plots.plot_two_groups(df_std_pre_ctrl, df_std_post_ctrl, best_channels, [2, 5], title = "[STANDARD] PRE-POST: CTRL", l = ["PRE", "POST"], c = ["g","brown"], size = (18, 7))
Plots.plot_two_groups(df_std_pre_plcb, df_std_post_plcb, best_channels, [2, 5], title = "[STANDARD] PRE-POST: PLCB", l = ["PRE", "POST"], c = ["g","brown"], size = (18, 7))
Plots.plot_two_groups(df_std_pre_exp, df_std_post_exp, best_channels, [2, 5], title = "[STANDARD] PRE-POST: EXP", l = ["PRE", "POST"], c = ["g","brown"], size = (18, 7))

-) **Controles**

Al comparar ambos instantes, podemos observar que existe un desplazamiento de la componente P2. El máximo se alcanza más tarde.


-) **Placebos**

Al comparar ambos instantes, podemos observar que existe un desplazamiento de la componente P2. EL máximo se alcanza antes.

-) **Experimentales**

Al comparar ambos instantes dentro de los experimentales, no se aprecian diferencias relevantes.

-) **Conclusión**

Estamos comparando estímulos *standard* entre PRE y POST. Estos no dejan de ser tareas automáticas que los sujetos van a acabar ignorando hasta detectar los *target*.

### (-) Target vs Standard

- PRE

In [ ]:
Plots.plot_two_groups(df_std_pre_ctrl, df_tgt_pre_ctrl, best_channels, [2, 5], title = "[PRE] CTRL: TGT vs STD", l = ["Standard", "Target"], c = ["b","r"], line = ["-","--"], size = (18, 7))
Plots.plot_two_groups(df_std_pre_plcb, df_tgt_pre_plcb, best_channels, [2, 5], title = "[PRE] PLCB: TGT vs STD", l = ["Standard", "Target"], c = ["b","r"], line = ["-","--"], size = (18, 7))
Plots.plot_two_groups(df_std_pre_exp, df_tgt_pre_exp, best_channels, [2, 5], title = "[PRE] EXP: TGT vs STD", l = ["Standard", "Target"], c = ["b","r"], line = ["-","--"], size = (18, 7))

- POST

In [ ]:
Plots.plot_two_groups(df_std_post_ctrl, df_tgt_post_ctrl, best_channels, [2, 5], title = "[POST] CTRL: TGT vs STD", l = ["Standard", "Target"], c = ["b","r"], line = ["-","--"], size = (18, 7))
Plots.plot_two_groups(df_std_post_plcb, df_tgt_post_plcb, best_channels, [2, 5], title = "[POST] PLCB: TGT vs STD", l = ["Standard", "Target"], c = ["b","r"], line = ["-","--"], size = (18, 7))
Plots.plot_two_groups(df_std_post_exp, df_tgt_post_exp, best_channels, [2, 5], title = "[POST] EXP: TGT vs STD", l = ["Standard", "Target"], c = ["b","r"], line = ["-","--"], size = (18, 7))

-------------------

## Cerebral-bands: $\delta$ - $\theta$ - $\alpha$ - $\beta$

La descomposición de los estímulos en bandas cerebrales permite analizar las variaciones de la actividad neuronal a diferentes niveles de frecuencia, proporcionando una visión detallada de la respuesta al estímulo. Este enfoque permite identificar patrones específicos de activación neuronal entre grupos e instantes según la frecuencia analizada, facilitando la búsqueda para análisis posteriores. Para ello, hemos aplicado *PassBand Filters*, que segmentan la señal en distintos rangos, como podemos ver a continuación. 

$$
\begin{array}{|c|c|}
\hline
\textbf{Rango de Frecuencia} & \textbf{Banda (Hz)} \\
\hline
\text{Delta ($\delta$)} & 1 - 4 \\
\hline
\text{Theta ($\theta$)} & 4 - 8 \\
\hline
\text{Alpha ($\alpha$)} & 8 - 12 \\
\hline
\text{Beta ($\beta$)} & 12 - 30 \\
\hline
\end{array}
$$


### **Delta**

#### (-) Target

- Groups contrast

In [ ]:
Plots.plot_band_three_groups(df_tgt_pre_ctrl, df_tgt_pre_plcb, df_tgt_pre_exp, best_channels, band = "delta", params = [2,5], title = "[Target - Delta] PRE: CTRL, PLCB and EXP", l = ["CTRL", "PLCB", "EXP"], c = ["b","black","r"])
Plots.plot_band_two_groups(df_tgt_post_ctrl, df_tgt_post_exp, best_channels, band = "delta", params = [2,5], title = "[Target - Delta] POST: CTRL vs EXP", l = ["CTRL", "EXP"], c = ["b","r"], line = ["-","-"])
Plots.plot_band_two_groups(df_tgt_post_plcb, df_tgt_post_exp, best_channels, band = "delta", params = [2,5], title = "[Target - Delta] POST: PLCB vs EXP", l = ["PLCB", "EXP"], c = ["black","r"], line = ["-","-"])

- Instants contrast

In [ ]:
Plots.plot_band_two_groups(df_tgt_pre_ctrl, df_tgt_post_ctrl, best_channels, band = "delta", params = [2, 5], title = "[Target - Delta] CTRL: PRE-POST", l = ["PRE", "POST"], c = ["g","brown"], size = (18, 7))
Plots.plot_band_two_groups(df_tgt_pre_plcb, df_tgt_post_plcb, best_channels, band = "delta", params = [2, 5], title = "[Target - Delta] PLCB: PRE-POST", l = ["PRE", "POST"], c = ["g","brown"], size = (18, 7))
Plots.plot_band_two_groups(df_tgt_pre_exp, df_tgt_post_exp, best_channels, band = "delta", params = [2, 5], title = "[Target - Delta] EXP: PRE-POST", l = ["PRE", "POST"], c = ["g","brown"], size = (18, 7))

#### (-) Standard

- Groups contrast

In [ ]:
Plots.plot_band_three_groups(df_std_pre_ctrl, df_std_pre_plcb, df_std_pre_exp, best_channels, band = "delta", params = [2,5], title = "[Standard - Delta] PRE: CTRL, PLCB and EXP", l = ["CTRL", "PLCB", "EXP"], c = ["b","black","r"], line = ["-","-","-"])
Plots.plot_band_two_groups(df_std_post_ctrl, df_std_post_exp, best_channels, band = "delta", params = [2,5], title = "[Standard - Delta] POST: CTRL vs EXP", l = ["CTRL", "EXP"], c = ["b","r"], line = ["-","-"])
Plots.plot_band_two_groups(df_std_post_plcb, df_std_post_exp, best_channels, band = "delta", params = [2,5], title = "[Standard - Delta] POST: PLCB vs EXP", l = ["PLCB", "EXP"], c = ["black","r"], line = ["-","-"])

- Instants contrast

In [ ]:
Plots.plot_band_two_groups(df_std_pre_ctrl, df_std_post_ctrl, best_channels, band = "delta", params = [2, 5], title = "[Standard - Delta] CTRL: PRE-POST", l = ["PRE", "POST"], c = ["g","brown"], size = (18, 7))
Plots.plot_band_two_groups(df_std_pre_plcb, df_std_post_plcb, best_channels, band = "delta", params = [2, 5], title = "[Standard - Delta] PLCB: PRE-POST", l = ["PRE", "POST"], c = ["g","brown"], size = (18, 7))
Plots.plot_band_two_groups(df_std_pre_exp, df_std_post_exp, best_channels, band = "delta", params = [2, 5], title = "[Standard - Delta] EXP: PRE-POST", l = ["PRE", "POST"], c = ["g","brown"], size = (18, 7))

### **Theta**

#### (-) Target

- Groups contrast

In [ ]:
Plots.plot_band_three_groups(df_tgt_pre_ctrl, df_tgt_pre_plcb, df_tgt_pre_exp, best_channels, band = "theta", params = [2,5], title = "[Target - Theta] PRE: CTRL, PLCB and EXP", l = ["CTRL", "PLCB", "EXP"], c = ["b","black","r"])
Plots.plot_band_two_groups(df_tgt_post_ctrl, df_tgt_post_exp, best_channels, band = "theta", params = [2,5], title = "[Target - Theta] POST: CTRL vs EXP", l = ["CTRL", "EXP"], c = ["b","r"], line = ["-","-"])
Plots.plot_band_two_groups(df_tgt_post_plcb, df_tgt_post_exp, best_channels, band = "theta", params = [2,5], title = "[Target - Theta] POST: PLCB vs EXP", l = ["PLCB", "EXP"], c = ["black","r"], line = ["-","-"])

- Instants contrast

In [ ]:
Plots.plot_band_two_groups(df_tgt_pre_ctrl, df_tgt_post_ctrl, best_channels, band = "theta", params = [2, 5], title = "[Target - Theta] CTRL: PRE-POST", l = ["PRE", "POST"], c = ["g","brown"], size = (18, 7))
Plots.plot_band_two_groups(df_tgt_pre_plcb, df_tgt_post_plcb, best_channels, band = "theta", params = [2, 5], title = "[Target - Theta] PLCB: PRE-POST", l = ["PRE", "POST"], c = ["g","brown"], size = (18, 7))
Plots.plot_band_two_groups(df_tgt_pre_exp, df_tgt_post_exp, best_channels, band = "theta", params = [2, 5], title = "[Target - Theta] EXP: PRE-POST", l = ["PRE", "POST"], c = ["g","brown"], size = (18, 7))

#### (-) Standard 

- Groups contrast

In [ ]:
Plots.plot_band_three_groups(df_std_pre_ctrl, df_std_pre_plcb, df_std_pre_exp, best_channels, band = "theta", params = [2,5], title = "[Standard - Theta] PRE: CTRL, PLCB and EXP", l = ["CTRL", "PLCB", "EXP"], c = ["b","black","r"])

Plots.plot_band_two_groups(df_std_post_plcb, df_std_post_exp, best_channels, band = "theta", params = [2,5], title = "[Standard - Theta] POST: PLCB vs EXP", l = ["PLCB", "EXP"], c = ["black","r"], line = ["-","-"])
Plots.plot_band_two_groups(df_std_post_ctrl, df_std_post_exp, best_channels, band = "theta", params = [2,5], title = "[Standard - Theta] POST: CTRL vs EXP", l = ["CTRL", "EXP"], c = ["b","r"], line = ["-","-"])

- Instants contrast

In [ ]:
Plots.plot_band_two_groups(df_std_pre_ctrl, df_std_post_ctrl, best_channels, band = "theta", params = [2, 5], title = "[Standard - Theta] CTRL: PRE-POST", l = ["PRE", "POST"], c = ["g","brown"], size = (18, 7))
Plots.plot_band_two_groups(df_std_pre_plcb, df_std_post_plcb, best_channels, band = "theta", params = [2, 5], title = "[Standard - Theta] PLCB: PRE-POST", l = ["PRE", "POST"], c = ["g","brown"], size = (18, 7))
Plots.plot_band_two_groups(df_std_pre_exp, df_std_post_exp, best_channels, band = "theta", params = [2, 5], title = "[Standard] - Theta] EXP: PRE-POST", l = ["PRE", "POST"], c = ["g","brown"], size = (18, 7))

### **Alpha**

#### (-) Target

- Groups contrast

In [ ]:
Plots.plot_band_three_groups(df_tgt_pre_ctrl, df_tgt_pre_plcb, df_tgt_pre_exp, best_channels, band = "alpha", params = [2,5], title = "[Target - Alpha] PRE: CTRL, PLCB and EXP", l = ["CTRL", "PLCB", "EXP"], c = ["b","black","r"])
Plots.plot_band_two_groups(df_tgt_post_ctrl, df_tgt_post_exp, best_channels, band = "alpha", params = [2,5], title = "[Target - Alpha] POST: CTRL vs EXP", l = ["CTRL", "EXP"], c = ["b","r"], line = ["-","-"])
Plots.plot_band_two_groups(df_tgt_post_plcb, df_tgt_post_exp, best_channels, band = "alpha", params = [2,5], title = "[Target - Alpha] POST: PLCB vs EXP", l = ["PLCB", "EXP"], c = ["black","r"], line = ["-","-"])

- Instants contrast

In [ ]:
Plots.plot_band_two_groups(df_tgt_pre_ctrl, df_tgt_post_ctrl, best_channels, band = "alpha", params = [2, 5], title = "[Target - Alpha] CTRL: PRE-POST", l = ["PRE", "POST"], c = ["g","brown"], size = (18, 7))
Plots.plot_band_two_groups(df_tgt_pre_plcb, df_tgt_post_plcb, best_channels, band = "alpha", params = [2, 5], title = "[Target - Alpha] PLCB: PRE-POST", l = ["PRE", "POST"], c = ["g","brown"], size = (18, 7))
Plots.plot_band_two_groups(df_tgt_pre_exp, df_tgt_post_exp, best_channels, band = "alpha", params = [2, 5], title = "[Target - Alpha] EXP: PRE-POST", l = ["PRE", "POST"], c = ["g","brown"], size = (18, 7))

#### (-) Standard

- Groups contrast

In [ ]:
Plots.plot_band_three_groups(df_std_pre_ctrl, df_std_pre_plcb, df_std_pre_exp, best_channels, band = "alpha", params = [2,5], title = "[Standard - Alpha] PRE: CTRL, PLCB and EXP", l = ["CTRL", "PLCB", "EXP"], c = ["b","black","r"])
Plots.plot_band_two_groups(df_std_post_plcb, df_std_post_exp, best_channels, band = "alpha", params = [2,5], title = "[Standard - Alpha] POST: PLCB vs EXP", l = ["PLCB", "EXP"], c = ["black","r"], line = ["-","-"])
Plots.plot_band_two_groups(df_std_post_ctrl, df_std_post_exp, best_channels, band = "alpha", params = [2,5], title = "[Standard - Alpha] POST: CTRL vs EXP", l = ["CTRL", "EXP"], c = ["b","r"], line = ["-","-"])

- Instants contrast

In [ ]:
Plots.plot_band_two_groups(df_std_pre_ctrl, df_std_post_ctrl, best_channels, band = "alpha", params = [2, 5], title = "[Standard - Alpha] CTRL: PRE-POST", l = ["PRE", "POST"], c = ["g","brown"], size = (18, 7))
Plots.plot_band_two_groups(df_std_pre_plcb, df_std_post_plcb, best_channels, band = "alpha", params = [2, 5], title = "[Standard - Alpha] PLCB: PRE-POST", l = ["PRE", "POST"], c = ["g","brown"], size = (18, 7))
Plots.plot_band_two_groups(df_std_pre_exp, df_std_post_exp, best_channels, band = "alpha", params = [2, 5], title = "[Standard] - Alpha] EXP: PRE-POST", l = ["PRE", "POST"], c = ["g","brown"], size = (18, 7))

### **Beta**

#### (-) Target

- Groups contrast

In [ ]:
Plots.plot_band_three_groups(df_tgt_pre_ctrl, df_tgt_pre_plcb, df_tgt_pre_exp, best_channels, band = "beta", params = [2,5], title = "[Target - Beta] PRE: CTRL, PLCB and EXP", l = ["CTRL", "PLCB", "EXP"], c = ["b","black","r"])
Plots.plot_band_two_groups(df_tgt_post_ctrl, df_tgt_post_exp, best_channels, band = "beta", params = [2,5], title = "[Target - Beta] POST: CTRL vs EXP", l = ["CTRL", "EXP"], c = ["b","r"], line = ["-","-"])
Plots.plot_band_two_groups(df_tgt_post_plcb, df_tgt_post_exp, best_channels, band = "beta", params = [2,5], title = "[Target - Beta] POST: PLCB vs EXP", l = ["PLCB", "EXP"], c = ["black","r"], line = ["-","-"])

- Instants contrast

In [ ]:
Plots.plot_band_two_groups(df_tgt_pre_ctrl, df_tgt_post_ctrl, best_channels, band = "beta", params = [2, 5], title = "[Target - Beta] CTRL: PRE-POST", l = ["PRE", "POST"], c = ["g","brown"], size = (18, 7))
Plots.plot_band_two_groups(df_tgt_pre_plcb, df_tgt_post_plcb, best_channels, band = "beta", params = [2, 5], title = "[Target - Beta] PLCB: PRE-POST", l = ["PRE", "POST"], c = ["g","brown"], size = (18, 7))
Plots.plot_band_two_groups(df_tgt_pre_exp, df_tgt_post_exp, best_channels, band = "beta", params = [2, 5], title = "[Target - Beta] EXP: PRE-POST", l = ["PRE", "POST"], c = ["g","brown"], size = (18, 7))

#### (-) Standard

- Groups contrast

In [ ]:
Plots.plot_band_three_groups(df_std_pre_ctrl, df_std_pre_plcb, df_std_pre_exp, best_channels, band = "beta", params = [2,5], title = "[Standard - Beta] PRE: CTRL, PLCB and EXP", l = ["CTRL", "PLCB", "EXP"], c = ["b","black","r"])

Plots.plot_band_two_groups(df_std_post_plcb, df_std_post_exp, best_channels, band = "beta", params = [2,5], title = "[Standard - Beta] POST: PLCB vs EXP", l = ["PLCB", "EXP"], c = ["black","r"], line = ["-","-"])
Plots.plot_band_two_groups(df_std_post_ctrl, df_std_post_exp, best_channels, band = "beta", params = [2,5], title = "[Standard - Beta] POST: CTRL vs EXP", l = ["CTRL", "EXP"], c = ["b","r"], line = ["-","-"])

- Instants contrast

In [ ]:
Plots.plot_band_two_groups(df_std_pre_ctrl, df_std_post_ctrl, best_channels, band = "beta", params = [2, 5], title = "[Standard - Beta] CTRL: PRE-POST", l = ["PRE", "POST"], c = ["g","brown"], size = (18, 7))
Plots.plot_band_two_groups(df_std_pre_plcb, df_std_post_plcb, best_channels, band = "beta", params = [2, 5], title = "[Standard - Beta] PLCB: PRE-POST", l = ["PRE", "POST"], c = ["g","brown"], size = (18, 7))
Plots.plot_band_two_groups(df_std_pre_exp, df_std_post_exp, best_channels, band = "beta", params = [2, 5], title = "[Standard] - Beta] EXP: PRE-POST", l = ["PRE", "POST"], c = ["g","brown"], size = (18, 7))

---------------

# Features

En esta sección, realizaremos un análisis estadístico con la **media** y **varianza** de las señales. Sin embargo, es importante tener en cuenta que las **características** han sido **extraídas de la ventana mediana** de cada uno de los 20 canales que posee cada sujeto. Más adelante, repetiremos el análisis promediando las características de cada ventana para garantizar una mayor consistencia.

**Objetivo**: 
- Analizar las ventanas en el rango de [-200, 700] ms a través de la media y la varianza por cada canal. 
- Cada punto en las gráficas representa un sujeto en un canal específico.
- Al graficar estas características, buscamos identificar posibles patrones o diferencias entre grupos o instantes.

**Consideraciones**: 
- Si bien los análisis previos sugieren que las **diferencias más relevantes** pueden encontrarse en **componentes específicas**, esta visualización nos dará una primera aproximación. Luego, aplicaremos análisis estadísticos para contrastar las diferencias que hemos detectado de forma visual. 


### Visualize channels

Al graficar la distribución de los sujetos en estímulos *target* y *standard*, hay dificultades para analizar correctamente la disposición espacial de los puntos. Hemos decidido apartar los sujetos con valores extremos y, así, tener una imagen limpia que refleje patrones o diferencias entre grupos, instantes o canales. 

Consideraciones:
- Estos sujetos solo son removidos para una visualización más clara. En los análisis posteriores se emplean todos los datos disponibles.

In [ ]:
path_tgt = "E:/TFM/CLUSTERING_ALL_ICA_by_segments/ODDBALL/features_tgt.csv"
path_std = "E:/TFM/CLUSTERING_ALL_ICA_by_segments/ODDBALL/features_std.csv"

df_feat_tgt = pd.read_csv(path_tgt, header=0, delimiter=';')
df_feat_std = pd.read_csv(path_std, header=0, delimiter=';')

channels = df_feat_tgt["ch"].unique()

grupos = ["CONTROL","PLCB","EXP"]
best_channels = list(set(channels) ^ set(['P8','T8', 'P4', 'O1', 'Oz', 'O2', 'Pz', 'P3', 'T7', 'P7']))  # Operador XOR (^) en conjuntos

#### (-) Target

In [ ]:
# Removemos sujetos con valores extremos
df_feat_tgt = Utils.remove_subjects(df_feat_tgt, [10,11,12,29,44,45,60,73,76])

# Normalizamos agrupando por componentes, ya que son datos diferentes
df_feat_tgt_norm = normalize(df_feat_tgt, components)

# Seleccionamos instantes PRE y POST para caracteristicas de estimulos Target
df_feat_tgt_pre, df_feat_tgt_post = Utils.group_by(df_feat_tgt_norm, col="instante", val=["PRE","POST"])

# Seleccionamos grupos CTRL, PLCB y EXP para caracteristicas de estimulos Target
df_feat_tgt_ctrl, df_feat_tgt_plcb, df_feat_tgt_exp = Utils.group_by(df_feat_tgt_norm, col="grupo", val=grupos)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
comp = "all"

# Configurar figura y ejes (4x4)
fig, axes = plt.subplots(2, 4, figsize=(18, 10))
fig.suptitle("Comparación PRE vs POST por Región EEG (Mean)", fontsize=16, y=1.02)

# Aplanar ejes para iterar fácilmente
axes = axes.flatten()

# Iterar sobre cada región EEG
for i, (region_name, channels) in enumerate(eeg_regions.items()):
    ax = axes[i]
    
    # Filtrar datos PRE y POST para la región actual
    region_pre = df_feat_tgt_pre[(df_feat_tgt_pre["ch"].isin(channels)) & (df_feat_tgt_pre["comp"] == comp)]
    region_post = df_feat_tgt_post[(df_feat_tgt_post["ch"].isin(channels)) & (df_feat_tgt_post["comp"] == comp)]
    
    # Asegurar que los IDs coincidan
    region_pre, region_post = Utils.select_common_ids(region_pre, region_post)
    
    # Calcular promedios por grupo
    avg_pre, avg_post = Utils.average_by_feature(data = [region_pre, region_post], features = ["mean"])
    ctrl_pre, plcb_pre, exp_pre = Utils.group_by(avg_pre, col="grupo", val=["CONTROL", "PLCB", "EXP"])
    ctrl_post, plcb_post, exp_post = Utils.group_by(avg_post, col="grupo", val=["CONTROL", "PLCB", "EXP"])
    
    # Scatter plot por grupo
    ax.scatter(ctrl_pre["mean"], ctrl_post["mean"], color='blue', label="CONTROL", alpha=0.7)
    ax.scatter(plcb_pre["mean"], plcb_post["mean"], color='gray', label="PLCB", alpha=0.7)
    ax.scatter(exp_pre["mean"], exp_post["mean"], color='red', label="EXP", alpha=0.7)
    
    # Ajustes del subplot
    ax.set_title(region_name, fontsize=12)
    ax.set_xlabel("PRE (mean)", fontsize=10)
    ax.set_ylabel("POST (mean)", fontsize=10)
    ax.grid(linestyle='--', alpha=0.3)
    ax.legend(fontsize=8)

# Ocultar ejes vacíos si hay menos de 16 regiones
for j in range(i + 1, len(axes)):
    axes[j].axis("off")

plt.tight_layout()
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

comp = "all"

# Configurar figura y ejes (4x4)
fig, axes = plt.subplots(2, 4, figsize=(18, 10))
fig.suptitle("Comparación PRE vs POST por Región EEG (Std)", fontsize=16, y=1.02)

# Aplanar ejes para iterar fácilmente
axes = axes.flatten()

# Iterar sobre cada región EEG
for i, (region_name, channels) in enumerate(eeg_regions.items()):
    ax = axes[i]
    
    # Filtrar datos PRE y POST para la región actual
    region_pre = df_feat_tgt_pre[(df_feat_tgt_pre["ch"].isin(channels)) & (df_feat_tgt_pre["comp"] == comp)]
    region_post = df_feat_tgt_post[(df_feat_tgt_post["ch"].isin(channels)) & (df_feat_tgt_post["comp"] == comp)]
    
    # Asegurar que los IDs coincidan
    region_pre, region_post = Utils.select_common_ids(region_pre, region_post)
    
    # Calcular promedios por grupo
    avg_pre, avg_post = Utils.average_by_feature(data = [region_pre, region_post], features = ["std"])
    ctrl_pre, plcb_pre, exp_pre = Utils.group_by(avg_pre, col="grupo", val=["CONTROL", "PLCB", "EXP"])
    ctrl_post, plcb_post, exp_post = Utils.group_by(avg_post, col="grupo", val=["CONTROL", "PLCB", "EXP"])
    
    # Scatter plot por grupo
    ax.scatter(ctrl_pre["std"], ctrl_post["std"], color='blue', label="CONTROL", alpha=0.7)
    ax.scatter(plcb_pre["std"], plcb_post["std"], color='gray', label="PLCB", alpha=0.7)
    ax.scatter(exp_pre["std"], exp_post["std"], color='red', label="EXP", alpha=0.7)
    
    # Ajustes del subplot
    ax.set_title(region_name, fontsize=12)
    ax.set_xlabel("PRE (std)", fontsize=10)
    ax.set_ylabel("POST (std)", fontsize=10)
    ax.grid(linestyle='--', alpha=0.3)
    ax.legend(fontsize=8)

# Ocultar ejes vacíos si hay menos de 16 regiones
for j in range(i + 1, len(axes)):
    axes[j].axis("off")

plt.tight_layout()
plt.show()

- Groups contrast

In [ ]:
Plots.plot_groups_by_features(df_feat_tgt_post, best_channels, group = ["CONTROL", "EXP"], params = [2,5], title = "[Target] POST: CTRL vs EXP", l = ["CTRL", "EXP"], c = ["b", "r"], size = (18, 10), b_id = False)
Plots.plot_groups_by_features(df_feat_tgt_post, best_channels, group = ["PLCB", "EXP"], params = [2,5], title = "[Target] POST: PLCB vs EXP", l = ["PLCB", "EXP"], c = ["black", "r"], size = (18, 10), b_id = False)

- Instants contrast

In [ ]:
Plots.plot_instants_by_features(df_feat_tgt_ctrl, best_channels, instant = ["PRE", "POST"], params = [2,5], title = "[Target] CTRL: PRE vs POST", l = ["PRE", "POST"], c = ["green", "brown"], size = (18, 10))
Plots.plot_instants_by_features(df_feat_tgt_plcb, best_channels, instant = ["PRE", "POST"], params = [2,5], title = "[Target] PLCB: PRE vs POST", l = ["PRE", "POST"], c = ["green", "brown"], size = (18, 10))
Plots.plot_instants_by_features(df_feat_tgt_exp, best_channels, instant = ["PRE", "POST"], params = [2,5], title = "[Target] EXP: PRE vs POST", l = ["PRE", "POST"], c = ["green", "brown"], size = (18, 10))

#### (-) Standard

In [ ]:
# Removemos sujetos con valores extremos
df_feat_std = Utils.remove_subjects(df_feat_std, [26,29,38,78])

# Normalizamos agrupando por componentes, ya que son datos diferentes
df_feat_std_norm = normalize(df_feat_std, components)

# Seleccionamos instantes PRE y POST para caracteristicas de estimulos Standard
df_feat_std_pre, df_feat_std_post = Utils.group_by(df_feat_std_norm, col="instante", val=["PRE","POST"])

# Seleccionamos grupos CTRL, PLCB y EXP para caracteristicas de estimulos Standard
df_feat_std_ctrl, df_feat_std_plcb, df_feat_std_exp = Utils.group_by(df_feat_std_norm, col="grupo", val=["CONTROL", "PLCB", "EXP"])

- Groups contrast

In [ ]:
Plots.plot_groups_by_features(df_feat_std_pre, best_channels, group = ["CONTROL", "PLCB", "EXP"], params = [4,5], title = "[Standard] PRE: CTRL, PLCB and EXP", l = ["CTRL", "PLCB", "EXP"], c = ["b", "black", "r"], size = (18, 15), b_id = False)
Plots.plot_groups_by_features(df_feat_std_post, best_channels, group = ["CONTROL", "PLCB", "EXP"], params = [4,5], title = "[Standard] POST: CTRL, PLCB and EXP", l = ["CTRL", "PLCB", "EXP"], c = ["b", "black", "r"], size = (18, 15), b_id = False)

- Instants contrast

In [ ]:
Plots.plot_instants_by_features(df_feat_std_ctrl, best_channels, instant = ["PRE", "POST"], params = [2,5], title = "[Standard] CTRL: PRE vs POST", l = ["PRE", "POST"], c = ["green", "brown"], size = (18, 10))
Plots.plot_instants_by_features(df_feat_std_plcb, best_channels, instant = ["PRE", "POST"], params = [2,5], title = "[Standard] PLCB: PRE vs POST", l = ["PRE", "POST"], c = ["green", "brown"], size = (18, 10))
Plots.plot_instants_by_features(df_feat_std_exp, best_channels, instant = ["PRE", "POST"], params = [2,5], title = "[Standard] EXP: PRE vs POST", l = ["PRE", "POST"], c = ["green", "brown"], size = (18, 10))

## Statistical analysis

A partir de las características **media** y **varianza**, vamos a comparar si existen diferencias significativas entre instantes dentro de cada grupo. Lo ideal sería obtener resultados consistentes con las gráficas que obtuvimos en las primeras secciones. Nuestra hipótesis sugiere que las diferencias se deberían presentar principalmente en las componentes ERP. 

- Estos resultados nos permitirán profundizar en el entendimiento del comportamiento de los sujetos bajo diferentes intervenciones. 
- Como primera aproximación, demostraríamos que existen patrones discriminantes entre instantes dentro de cada grupo.

Los contrastes se realizan de la siguiente manera:

- Dentro de un mismo grupo, seleccionamos la componente P3, por ejemplo, y comparamos todos los sujetos en el instante PRE frente al POST. 
- Para la comparación, tomamos en cuenta todos los canales del sujeto, es decir, construimos distribuciones para PRE y POST que incluyen toda la variabilidad.
- Aplicamos una prueba estadística no paramétrica de Wilcoxon y una corrección por el método de False Discovery Rate (Benjamini-Hochberg).

In [ ]:
# Ruta para archivos con características de estímulos Target (tgt) y Standard (std)
path_tgt = "E:/TFM/CLUSTERING_ALL_ICA_by_segments/ODDBALL/features_tgt.csv"
path_std = "E:/TFM/CLUSTERING_ALL_ICA_by_segments/ODDBALL/features_std.csv"

# Cargamos ambos archivos en forma de DataFrames
df_feat_tgt = pd.read_csv(path_tgt, header=0, delimiter=';')
df_feat_std = pd.read_csv(path_std, header=0, delimiter=';')

# Informacion sobre los DataFrames
info = df_feat_tgt.columns[:6].to_list()
channels = df_feat_tgt["ch"].unique().tolist()
components = df_feat_tgt["comp"].unique().tolist()
grupos = ["CONTROL", "PLCB", "EXP"]
features = ["mean","std"]

eeg_regions = {
    "Frontal": ["Fp1", "Fp2", "F3", "Fz", "F4", "F7", "F8"],
    "Central": ["C3", "Cz", "C4"],
    "Parietal": ["P3", "Pz", "P4"],
    "Occipital": ["O1", "Oz", "O2"],
    "Temporal": ["T7", "T8"],
    "Parieto-temporal": ["P7", "P8"],
    "Parieto-occipital": ["P3", "P4"],
    "Linea-media":["Pz","Cz","Oz"],
    "All":channels
}

# Display info
print("Header with info:",info)
print("Channels:",channels)
print("Components:",components)
print("Features:", features)

In [ ]:
def compute_paired_test(pre, post, feat, channels, flag="wilcoxon"):
    
    import scipy.stats as stats
    
    combined_dict = {key: [pre[key], post[key]] for key in pre}

    comparaciones = []
    for grupo, (pre, post) in combined_dict.items():
        p_values = []  
        for ch in channels:
        #for region, ch_group in eeg_regions.items():

            # Seleccionamos los electrodos que conforman cada region cerebral
            post_data = post[post['ch'].isin([ch])]
            pre_data = pre[pre['ch'].isin([ch])]

            # Promediamos los canales por caracteristicas
            #avg_pre, avg_post = Utils.average_by_feature([pre_data, post_data], [feat])
            
            if pre_data[feat].mean() < post_data[feat].mean():
                contrast = "PRE < POST"
            else:
                contrast = "PRE > POST"

            if flag=="wilcoxon":
                t_stat, p_value = stats.wilcoxon(pre_data[feat], post_data[feat])

            else:
                t_stat, p_value = stats.ttest_rel(pre_data[feat], post_data[feat])

            p_values.append(p_value)

            comparaciones.append({
                "feat":feat,
                "grupo": grupo,
                "comp": pre["comp"].unique()[0],
                "region": ch,
                "contrast": contrast,
                "t_stat": t_stat,
                "p_value": p_value
            })
        

    return pd.DataFrame(comparaciones)

### (-) Target

In [ ]:
# Normalizamos agrupando por componentes, ya que son datos diferentes
df_feat_tgt_norm = normalize(df_feat_tgt, components)

-) **Mean**

In [ ]:
results = []
feat = "mean"

for df_comp in Utils.group_by(df_feat_tgt_norm, col="comp", val=components):

    tgt_feat_pre, tgt_feat_post = Utils.group_by(df_comp, col="instante", val=["PRE","POST"])
    tgt_feat_pre, tgt_feat_post = Utils.select_common_ids(tgt_feat_pre, tgt_feat_post)

    feat_pre_ctrl, feat_pre_plcb, feat_pre_exp = Utils.group_by(tgt_feat_pre, col="grupo", val=grupos)
    feat_post_ctrl, feat_post_plcb, feat_post_exp = Utils.group_by(tgt_feat_post, col="grupo", val=grupos)

    pre = dict(zip(grupos, Utils.group_by(tgt_feat_pre, col="grupo", val=grupos)))
    post = dict(zip(grupos, Utils.group_by(tgt_feat_post, col="grupo", val=grupos)))

    df = compute_paired_test(pre, post, feat, channels, flag="wilcoxon")

    results.append(df)

df_tests = pd.concat(results).reset_index(drop=True)

from statsmodels.stats.multitest import multipletests

adj_results = []

for df in Utils.group_by(df_tests, col="grupo", val=grupos):

    for df_comp in Utils.group_by(df, col="comp", val=components):

        _,adj_p_val,_,_ = multipletests(df_comp["p_value"], method="fdr_bh")

        df_comp["adj_p_value"] = adj_p_val

        adj_results.append(df_comp)

df_adj_tests = pd.concat(adj_results).reset_index(drop=True)

# df_adj_tests[df_adj_tests["adj_p_value"]<0.05]
df_tests[df_tests["p_value"]<0.05]

-) **Standard Deviation**

In [ ]:
results = []
feat = "std"

for df_comp in Utils.group_by(df_feat_tgt_norm, col="comp", val=components):

    tgt_feat_pre, tgt_feat_post = Utils.group_by(df_comp, col="instante", val=["PRE","POST"])
    tgt_feat_pre, tgt_feat_post = Utils.select_common_ids(tgt_feat_pre, tgt_feat_post)

    feat_pre_ctrl, feat_pre_plcb, feat_pre_exp = Utils.group_by(tgt_feat_pre, col="grupo", val=grupos)
    feat_post_ctrl, feat_post_plcb, feat_post_exp = Utils.group_by(tgt_feat_post, col="grupo", val=grupos)

    pre = dict(zip(grupos, Utils.group_by(tgt_feat_pre, col="grupo", val=grupos)))
    post = dict(zip(grupos, Utils.group_by(tgt_feat_post, col="grupo", val=grupos)))

    df = compute_paired_test(pre, post, feat, channels, flag="wilcoxon")

    results.append(df)

df_tests = pd.concat(results).reset_index(drop=True)

from statsmodels.stats.multitest import multipletests

adj_results = []

for df in Utils.group_by(df_tests, col="grupo", val=grupos):

    for df_comp in Utils.group_by(df, col="comp", val=components):

        _,adj_p_val,_,_ = multipletests(df_comp["p_value"], method="fdr_bh")

        df_comp["adj_p_value"] = adj_p_val

        adj_results.append(df_comp)

df_adj_tests = pd.concat(adj_results).reset_index(drop=True)

df_adj_tests[df_adj_tests["adj_p_value"]<0.05]

### (-) Standard

In [ ]:
# Normalizamos agrupando por componentes, ya que son datos diferentes
df_feat_std_norm = normalize(df_feat_std, components)

-) **Mean**

In [ ]:
results = []
feat = "mean"

for df_comp in Utils.group_by(df_feat_std_norm, col="comp", val=components):

    tgt_feat_pre, tgt_feat_post = Utils.group_by(df_comp, col="instante", val=["PRE","POST"])
    tgt_feat_pre, tgt_feat_post = Utils.select_common_ids(tgt_feat_pre, tgt_feat_post)

    feat_pre_ctrl, feat_pre_plcb, feat_pre_exp = Utils.group_by(tgt_feat_pre, col="grupo", val=grupos)
    feat_post_ctrl, feat_post_plcb, feat_post_exp = Utils.group_by(tgt_feat_post, col="grupo", val=grupos)

    pre = dict(zip(grupos, Utils.group_by(tgt_feat_pre, col="grupo", val=grupos)))
    post = dict(zip(grupos, Utils.group_by(tgt_feat_post, col="grupo", val=grupos)))

    df = compute_paired_test(pre, post, feat, channels, flag="wilcoxon")

    results.append(df)

df_tests = pd.concat(results).reset_index(drop=True)

from statsmodels.stats.multitest import multipletests

adj_results = []

for df in Utils.group_by(df_tests, col="grupo", val=grupos):

    for df_comp in Utils.group_by(df, col="comp", val=components):

        _,adj_p_val,_,_ = multipletests(df_comp["p_value"], method="fdr_bh")

        df_comp["adj_p_value"] = adj_p_val

        adj_results.append(df_comp)

df_adj_tests = pd.concat(adj_results).reset_index(drop=True)

df_adj_tests[df_adj_tests["adj_p_value"]<0.05]

-) **Standard Deviation**

In [ ]:
results = []
feat = "std"

for df_comp in Utils.group_by(df_feat_std_norm, col="comp", val=components):

    tgt_feat_pre, tgt_feat_post = Utils.group_by(df_comp, col="instante", val=["PRE","POST"])
    tgt_feat_pre, tgt_feat_post = Utils.select_common_ids(tgt_feat_pre, tgt_feat_post)

    feat_pre_ctrl, feat_pre_plcb, feat_pre_exp = Utils.group_by(tgt_feat_pre, col="grupo", val=grupos)
    feat_post_ctrl, feat_post_plcb, feat_post_exp = Utils.group_by(tgt_feat_post, col="grupo", val=grupos)

    pre = dict(zip(grupos, Utils.group_by(tgt_feat_pre, col="grupo", val=grupos)))
    post = dict(zip(grupos, Utils.group_by(tgt_feat_post, col="grupo", val=grupos)))

    df = compute_paired_test(pre, post, feat, channels, flag="wilcoxon")

    results.append(df)

df_tests = pd.concat(results).reset_index(drop=True)

from statsmodels.stats.multitest import multipletests

adj_results = []

for df in Utils.group_by(df_tests, col="grupo", val=grupos):

    for df_comp in Utils.group_by(df, col="comp", val=components):

        _,adj_p_val,_,_ = multipletests(df_comp["p_value"], method="fdr_bh")

        df_comp["adj_p_value"] = adj_p_val

        adj_results.append(df_comp)

df_adj_tests = pd.concat(adj_results).reset_index(drop=True)

df_adj_tests[df_adj_tests["adj_p_value"]<0.05]

-------------


## Approximation entropy

La entropía de aproximación es una medida que nos dice cómo de predecible o caótico es un conjunto de datos. Se usa mucho en el análisis de señales biológicas (ondas cerebrales) y en series temporales en general.

Partimos de una serie temporal con diferentes fluctuaciones que reflejan actividad eléctrica. Si las fluctuaciones siguen un patró claro y repetitivo, la entropía de aproximación será baja, porque es fácil de predecir. En cambio, si los números parecen aleatorios y desordenados, la entropía de aproximación será alta, porque hay mucha incertidumbre sobre el siguiente valor.  

- Baja entropía de aproximación --> Más regularidad y menos caos.
- Alta entropía de aproximación --> Más aleatoriedad y menos predecible.

Este tipo de aproximación es una adaptación de la entropía de Shannon enfocada en series temporales.

¿Por qué usamos la distancia máxima?

La idea principal es asegurarnos de que todas las posiciones dentro de una subsecuencia sean similares para que dos patrones sean considerados equivalentes. Si tomáramos la diferencia promedio entre cada punto de las subsecuencias, podríamos perder información. Puede ser que la diferencia promedio enmascare parejas de puntos con una gran diferencia entre sí, lo que sugiere que esa pareja de secuencias no deberían ser consideradas similares.

- Si dos secuencias tienen una sola posición con una gran diferencia, no deberían considerarse equivalentes, ya que la señal no se está comportando de manera predecible.

Ejemplo con audio:

Si comparamos fragmentos de una canción y en dos fragmentos todas las notas son casi iguales excepto una, esa única diferencia puede hacer que el audio suene muy distinto. La distancia máxima nos ayuda a capturar esos cambios bruscos.

La **entropía de aproximación (ApEn)** se calcula como la diferencia entre dos funciones $\phi(m, r)$:


$$ApEn(m, r) = \phi(m, r) - \phi(m+1, r),\ where:$$


$$\phi(m, r) = \frac{1}{N - m + 1} \sum_{i=1}^{N-m+1} \log C^{m}_{i}$$

En esta fórmula:
- $m$ es el tamaño del patrón.
- $r$ es la tolerancia, generalmente un múltiplo de la desviación estándar de la serie.
- $C^{m}_{i}$ es la proporción de patrones similares de la secuencia $i$.
- $N$ es la longitud total de la serie de tiempo.

Ejemplos:
- *Stationarity assessment of resting state condition via permutation entropy on EEG recordings* [https://doi.org/10.1038/s41598-024-82089-0]
- *Entropy modulation of electroencephalographic signals in physiological aging* [https://doi.org/10.1016/j.mad.2021.111472]

In [ ]:
print("Header with info:",info)
print("Channels:",channels)
print("Components:",components)
grupos = ["CONTROL", "PLCB", "EXP"]
best_channels = list(set(channels) ^ set(['P8','T8', 'P4', 'O1', 'Oz', 'O2', 'Pz', 'P3', 'T7', 'P7']))  # Operador XOR (^) en conjuntos

### Full-Band window

Hay sujetos en los grupos con valores de entropía exageradamente altos. Deben ser outliers. Esa es la razón por la que en las gráficas aparecen los nodos con un rojo intenso y no sale significativo el test. No creo que removerlos sea buena idea. En el rango completo de frecuencias [Target-All] se ve bien el efecto. En los ritmos cerebrales hay posibles outliers.

- **Target**

In [ ]:
# Ruta del archivo que deseas comprobar
file_path = "apEn_tgt.csv"

# Comprobar si el archivo existe
if os.path.exists(file_path):
    print(f"El archivo '{file_path}' existe.")
else:
    print(f"El archivo '{file_path}' no existe.")
    print()

    # Guardamos todos los DataFrames en una lista
    list_dfs = [df_tgt_pre_ctrl, df_tgt_pre_plcb, df_tgt_pre_exp, df_tgt_post_ctrl, df_tgt_post_plcb, df_tgt_post_exp]

    # Calculamos y guardamos la entropia por aproximacion
    Entropy.get_apEn_dataframe(list_dfs, channels, components).to_csv("apEn_tgt.csv",index=False,sep=';')

# Cargamos los datos en un DataFrame
df_tgt_apEn = pd.read_csv("apEn_tgt.csv", header=0, delimiter=';')

# Seleccionamos la entropia de aproximacion por instante
apEn_pre, apEn_post = Utils.group_by(df_tgt_apEn[df_tgt_apEn['comp']=='all'], col="instante", val=["PRE","POST"])

# Seleccionamos la entropia de aproximacion por instante y de todo el estimulo (all).
apEn_pre_ctrl, apEn_pre_plcb, apEn_pre_exp = Utils.group_by(apEn_pre, col="grupo", val=grupos)
apEn_post_ctrl, apEn_post_plcb, apEn_post_exp = Utils.group_by(apEn_post, col="grupo", val=grupos)

def get_channels(df1, df0):
    # Definimos DataFrame para las diferencias 
    # de entropia (POST-PRE) de cada sujeto y canal
    df0_ch = pd.DataFrame([], columns=df0['ch'].unique())
    df1_ch = pd.DataFrame([], columns=df1['ch'].unique())

    for (n,id),df0_id in df0.groupby(['n_test','id']):

        # Seleccionamos las mismas filas en ambos DataFrames
        df1_id = df1[(df1['n_test']==n)&(df1['id']==id)]

        df0_ch.loc[len(df0_ch)] = df0_id['H'].values
        df1_ch.loc[len(df1_ch)] = df1_id['H'].values

    return df1_ch, df0_ch


post_ctrl, pre_ctrl = get_channels(apEn_post_ctrl, apEn_pre_ctrl)

post_plcb, pre_plcb = get_channels(apEn_post_plcb, apEn_pre_plcb)

post_exp, pre_exp = get_channels(apEn_post_exp, apEn_pre_exp)


import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

vmax = np.max([np.max(pre_ctrl), np.max(pre_plcb), np.max(pre_exp)])
vmin = np.min([np.min(pre_ctrl), np.min(pre_plcb), np.min(pre_exp)])
centro = np.median([np.median(pre_ctrl), np.median(pre_plcb), np.median(pre_exp)])

# Crear los heatmaps
fig, axes = plt.subplots(1, 3, figsize=(20, 5))

sns.heatmap(pre_ctrl, ax=axes[0], cmap="coolwarm", cbar=True, center=centro, vmax=vmax, vmin=vmin)
axes[0].set_title("Electrodos - Pre - CTRL")

sns.heatmap(post_ctrl, ax=axes[1], cmap="coolwarm", cbar=True,center=centro, vmax=vmax, vmin=vmin)
axes[1].set_title("Electrodos - Post - CTRL")

percent_change = (post_ctrl - pre_ctrl) / pre_ctrl  # Convertir a %

sns.barplot(x=pre_ctrl.columns, y=np.median(percent_change,axis=0)*100, ax=axes[2], color="steelblue")
axes[2].set_title("Cambio Mediano (%) por Electrodo")
axes[2].set_xlabel("Electrodos")
axes[2].set_ylabel("Cambio (%)")

plt.show()



fig, axes = plt.subplots(1, 3, figsize=(20, 5))

sns.heatmap(pre_plcb, ax=axes[0], cmap="coolwarm", cbar=True, center=centro, vmax=vmax, vmin=vmin)
axes[0].set_title("Electrodos - Pre - PLCB")

sns.heatmap(post_plcb, ax=axes[1], cmap="coolwarm", cbar=True,center=centro, vmax=vmax, vmin=vmin)
axes[1].set_title("Electrodos - Post - PLCB")

percent_change = (post_plcb - pre_plcb) / pre_plcb  # Convertir a %

sns.barplot(x=pre_plcb.columns, y=np.median(percent_change,axis=0)*100, ax=axes[2], color="steelblue")
axes[2].set_title("Cambio Mediano (%) por Electrodo")
axes[2].set_xlabel("Electrodos")
axes[2].set_ylabel("Cambio (%)")

plt.show()

fig, axes = plt.subplots(1, 3, figsize=(20, 5))

sns.heatmap(pre_exp, ax=axes[0], cmap="coolwarm", cbar=True, center=centro, vmax=vmax, vmin=vmin)
axes[0].set_title("Electrodos - Pre - EXP")

sns.heatmap(post_exp, ax=axes[1], cmap="coolwarm", cbar=True,center=centro, vmax=vmax, vmin=vmin)
axes[1].set_title("Electrodos - Post - EXP")
percent_change = (post_exp - pre_exp) / pre_exp  # Convertir a %

sns.barplot(x=pre_exp.columns, y=np.median(percent_change,axis=0)*100, ax=axes[2], color="steelblue")
axes[2].set_title("Cambio Mediano (%) por Electrodo")
axes[2].set_xlabel("Electrodos")
axes[2].set_ylabel("Cambio (%)")

plt.show()

# diff_plcb = Entropy.compute_diff(apEn_post_plcb, apEn_pre_plcb)

# diff_exp = Entropy.compute_diff(apEn_post_exp, apEn_pre_exp)


# # Calculamos la diferencia mediana por electrodo en cada grupo

# # Approximation entropy: [ctrl_array, plcb_array, exp_array]
# apEn_arrays = [diff_ctrl.median(), diff_plcb.median(), diff_exp.median()]

# # Ploteamos diferencias
# Entropy.plot_diff(apEn_arrays, electrodes, title = "Target-All", subtitles = ["CTRL","PLCB","EXP"])

# # Corremos estadisticos 
# dict_apEn = {'CTRL': [apEn_pre_ctrl, apEn_post_ctrl],
#              'PLCB': [apEn_pre_plcb, apEn_post_plcb],
#              'EXP': [apEn_pre_exp, apEn_post_exp]} 

# pre = dict(zip(grupos, Utils.group_by(apEn_pre, col="grupo", val=grupos)))
# post = dict(zip(grupos, Utils.group_by(apEn_post, col="grupo", val=grupos)))   

# # Guardamos los resultados de los test pareados
# stats_results = Entropy.compute_paired_test(pre, post, channels, "wilcoxon")

# stats_results[(stats_results['p_value']<0.05)]

In [ ]:
# comp = 'all'

# CTRL = {'PRE': Entropy.combine_channels(apEn_pre_ctrl, eeg_regions).values(),
#         'POST': Entropy.combine_channels(apEn_post_ctrl, eeg_regions).values()}

# PLCB = {'PRE': Entropy.combine_channels(apEn_pre_plcb, eeg_regions).values(),
#        'POST': Entropy.combine_channels(apEn_post_plcb, eeg_regions).values()}

# EXP = {'PRE': Entropy.combine_channels(apEn_pre_exp, eeg_regions).values(),
#        'POST': Entropy.combine_channels(apEn_post_exp, eeg_regions).values()}

# Entropy.plot_by_regions(CTRL, PLCB, EXP, eeg_regions.keys())

- Comparando los instantes PRE y POST dentro del grupo **CTRL**, podemos ver que su respuesta al estímulo se vuelve **más caótica e impredecible**.
- Comparando los instantes PRE y POST dentro del grupo **PLCB**, podemos ver que su respuesta al estímulo se vuelve **más ordenada y predecible**.
- Comparando los instantes PRE y POST dentro del grupo **EXP**, podemos ver que su respuesta al estímulo se mantiene **constante** en términos de entropía.

### Cerebral-bands: $\delta$ - $\theta$ - $\alpha$ - $\beta$

- **Target**

In [ ]:
# Ruta del archivo que deseas comprobar
file_paths = {"delta": "./cerebral_bands/apEn_tgt_delta.csv",
             "theta": "./cerebral_bands/apEn_tgt_theta.csv",
             "alpha": "./cerebral_bands/apEn_tgt_alpha.csv",
             "beta": "./cerebral_bands/apEn_tgt_beta.csv"}

# Parametros de cada banda a analizar
params = {"delta": {'lowcut':1,'highcut':4,'fs':500},
          "theta": {'lowcut':4,'highcut':8,'fs':500},
          "alpha": {'lowcut':8,'highcut':12,'fs':500},
          "beta": {'lowcut':12,'highcut':30,'fs':500}}

# Para cada banda de interes, then:
for band, path in file_paths.items():
    # Comprobar si el archivo existe
    if os.path.exists(path):
        print(f"El archivo '{path}' existe.")
    else:
        print(f"El archivo '{path}' no existe.")
        print()

        # Guardamos todos los DataFrames en una lista
        list_dfs = [df_tgt_pre_ctrl, df_tgt_pre_plcb, df_tgt_pre_exp, df_tgt_post_ctrl, df_tgt_post_plcb, df_tgt_post_exp]
        # Guardamos los valores de entropia en un DataFrame
        Entropy.get_apEn_dataframe(list_dfs, channels, ['all'], params[band]).to_csv(path, index=False, sep=';')
        print()

#### **Delta**

In [ ]:
# Cargamos los datos en un DataFrame
df_delta = pd.read_csv("./cerebral_bands/apEn_tgt_delta.csv", header=0, delimiter=';')

# Seleccionamos la entropia de aproximacion por instante
apEn_pre, apEn_post = Utils.group_by(df_delta[df_delta['comp']=='all'], col="instante", val=["PRE","POST"])

# Seleccionamos la entropia de aproximacion por instante y de todo el estimulo (all).
apEn_pre_ctrl, apEn_pre_plcb, apEn_pre_exp = Utils.group_by(apEn_pre, col="grupo", val=grupos)
apEn_post_ctrl, apEn_post_plcb, apEn_post_exp = Utils.group_by(apEn_post, col="grupo", val=grupos)



post_ctrl, pre_ctrl = get_channels(apEn_post_ctrl, apEn_pre_ctrl)

post_plcb, pre_plcb = get_channels(apEn_post_plcb, apEn_pre_plcb)

post_exp, pre_exp = get_channels(apEn_post_exp, apEn_pre_exp)


import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

vmax = np.max([np.max(pre_ctrl), np.max(pre_plcb), np.max(pre_exp)])
vmin = np.min([np.min(pre_ctrl), np.min(pre_plcb), np.min(pre_exp)])
centro = np.median([np.median(pre_ctrl), np.median(pre_plcb), np.median(pre_exp)])

# Crear los heatmaps
fig, axes = plt.subplots(1, 3, figsize=(20, 5))

sns.heatmap(pre_ctrl, ax=axes[0], cmap="coolwarm", cbar=True, center=centro, vmax=vmax, vmin=vmin)
axes[0].set_title("Electrodos - Pre")

sns.heatmap(post_ctrl, ax=axes[1], cmap="coolwarm", cbar=True,center=centro, vmax=vmax, vmin=vmin)
axes[1].set_title("Electrodos - Post")

percent_change = (post_ctrl - pre_ctrl) / pre_ctrl  # Convertir a %

sns.barplot(x=pre_ctrl.columns, y=np.median(percent_change,axis=0)*100, ax=axes[2], color="steelblue")
axes[2].set_title("Cambio Mediano (%) por Electrodo")
axes[2].set_xlabel("Electrodos")
axes[2].set_ylabel("Cambio (%)")

plt.show()



fig, axes = plt.subplots(1, 3, figsize=(20, 5))

sns.heatmap(pre_plcb, ax=axes[0], cmap="coolwarm", cbar=True, center=centro, vmax=vmax, vmin=vmin)
axes[0].set_title("Electrodos - Pre")

sns.heatmap(post_plcb, ax=axes[1], cmap="coolwarm", cbar=True,center=centro, vmax=vmax, vmin=vmin)
axes[1].set_title("Electrodos - Post")

percent_change = (post_plcb - pre_plcb) / pre_plcb  # Convertir a %

sns.barplot(x=pre_plcb.columns, y=np.median(percent_change,axis=0)*100, ax=axes[2], color="steelblue")
axes[2].set_title("Cambio Mediano (%) por Electrodo")
axes[2].set_xlabel("Electrodos")
axes[2].set_ylabel("Cambio (%)")

plt.show()

fig, axes = plt.subplots(1, 3, figsize=(20, 5))

sns.heatmap(pre_exp, ax=axes[0], cmap="coolwarm", cbar=True, center=centro, vmax=vmax, vmin=vmin)
axes[0].set_title("Electrodos - Pre")

sns.heatmap(post_exp, ax=axes[1], cmap="coolwarm", cbar=True,center=centro, vmax=vmax, vmin=vmin)
axes[1].set_title("Electrodos - Post")
percent_change = (post_exp - pre_exp) / pre_exp  # Convertir a %

sns.barplot(x=pre_exp.columns, y=np.median(percent_change,axis=0)*100, ax=axes[2], color="steelblue")
axes[2].set_title("Cambio Mediano (%) por Electrodo")
axes[2].set_xlabel("Electrodos")
axes[2].set_ylabel("Cambio (%)")

plt.show()



# # Calculamos la diferencia de apEn entre POST-PRE
# diff_ctrl = Entropy.compute_diff(apEn_post_ctrl, apEn_pre_ctrl)

# diff_plcb = Entropy.compute_diff(apEn_post_plcb, apEn_pre_plcb)

# diff_exp = Entropy.compute_diff(apEn_post_exp, apEn_pre_exp)

# # Calculamos la diferencia mediana por electrodo en cada grupo

# # Approximation entropy: [ctrl_array, plcb_array, exp_array]
# apEn_arrays = [diff_ctrl.median(), diff_plcb.median(), diff_exp.median()]

# # Ploteamos diferencias
# Entropy.plot_diff(apEn_arrays, electrodes, title = "Target-Delta", subtitles = ["CTRL","PLCB","EXP"])

# # Corremos estadisticos 
# dict_apEn = {'CTRL': [apEn_pre_ctrl, apEn_post_ctrl],
#              'PLCB': [apEn_pre_plcb, apEn_post_plcb],
#              'EXP': [apEn_pre_exp, apEn_post_exp]} 

# pre = dict(zip(grupos, Utils.group_by(apEn_pre, col="grupo", val=grupos)))
# post = dict(zip(grupos, Utils.group_by(apEn_post, col="grupo", val=grupos)))  

# # Guardamos los resultados de los test pareados
# stats_results = Entropy.compute_paired_test(pre, post, channels, "wilcoxon")

# stats_results[(stats_results['p_value']<0.05)]

In [ ]:
# comp = 'all'

# CTRL = {'PRE': Entropy.combine_channels(apEn_pre_ctrl, eeg_regions).values(),
#         'POST': Entropy.combine_channels(apEn_post_ctrl, eeg_regions).values()}

# PLCB = {'PRE': Entropy.combine_channels(apEn_pre_plcb, eeg_regions).values(),
#        'POST': Entropy.combine_channels(apEn_post_plcb, eeg_regions).values()}

# EXP = {'PRE': Entropy.combine_channels(apEn_pre_exp, eeg_regions).values(),
#        'POST': Entropy.combine_channels(apEn_post_exp, eeg_regions).values()}

# Entropy.plot_by_regions(CTRL, PLCB, EXP, eeg_regions.keys())

Al comparar los instantes PRE y POST, los 3 grupos presentan una respuesta al estímulo **más caótica e impredecible** en la banda de frecuencia **Delta**.
- PLCB > CTRL > EXP

#### **Theta**

In [ ]:
# Cargamos los datos en un DataFrame
df_theta = pd.read_csv("./cerebral_bands/apEn_tgt_theta.csv", header=0, delimiter=';')

# Seleccionamos la entropia de aproximacion por instante
apEn_pre, apEn_post = Utils.group_by(df_theta[df_theta['comp']=='all'], col="instante", val=["PRE","POST"])

# Seleccionamos la entropia de aproximacion por instante y de todo el estimulo (all).
apEn_pre_ctrl, apEn_pre_plcb, apEn_pre_exp = Utils.group_by(apEn_pre, col="grupo", val=grupos)
apEn_post_ctrl, apEn_post_plcb, apEn_post_exp = Utils.group_by(apEn_post, col="grupo", val=grupos)


def get_channels(df1, df0):
    # Definimos DataFrame para las diferencias 
    # de entropia (POST-PRE) de cada sujeto y canal
    df0_ch = pd.DataFrame([], columns=df0['ch'].unique())
    df1_ch = pd.DataFrame([], columns=df1['ch'].unique())

    for (n,id),df0_id in df0.groupby(['n_test','id']):

        # Seleccionamos las mismas filas en ambos DataFrames
        df1_id = df1[(df1['n_test']==n)&(df1['id']==id)]

        df0_ch.loc[len(df0_ch)] = df0_id['H'].values
        df1_ch.loc[len(df1_ch)] = df1_id['H'].values

    return df1_ch, df0_ch


post_ctrl, pre_ctrl = get_channels(apEn_post_ctrl, apEn_pre_ctrl)

post_plcb, pre_plcb = get_channels(apEn_post_plcb, apEn_pre_plcb)

post_exp, pre_exp = get_channels(apEn_post_exp, apEn_pre_exp)


import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

vmax = np.max([np.max(pre_ctrl), np.max(pre_plcb), np.max(pre_exp)])
vmin = np.min([np.min(pre_ctrl), np.min(pre_plcb), np.min(pre_exp)])
centro = np.median([np.median(pre_ctrl), np.median(pre_plcb), np.median(pre_exp)])

# Crear los heatmaps
fig, axes = plt.subplots(1, 3, figsize=(20, 5))

sns.heatmap(pre_ctrl, ax=axes[0], cmap="coolwarm", cbar=True, center=centro, vmax=vmax, vmin=vmin)
axes[0].set_title("Electrodos - Pre")

sns.heatmap(post_ctrl, ax=axes[1], cmap="coolwarm", cbar=True,center=centro, vmax=vmax, vmin=vmin)
axes[1].set_title("Electrodos - Post")

percent_change = (post_ctrl - pre_ctrl) / pre_ctrl  # Convertir a %

sns.barplot(x=pre_ctrl.columns, y=np.median(percent_change,axis=0)*100, ax=axes[2], color="steelblue")
axes[2].set_title("Cambio Mediano (%) por Electrodo")
axes[2].set_xlabel("Electrodos")
axes[2].set_ylabel("Cambio (%)")

plt.show()



fig, axes = plt.subplots(1, 3, figsize=(20, 5))

sns.heatmap(pre_plcb, ax=axes[0], cmap="coolwarm", cbar=True, center=centro, vmax=vmax, vmin=vmin)
axes[0].set_title("Electrodos - Pre")

sns.heatmap(post_plcb, ax=axes[1], cmap="coolwarm", cbar=True,center=centro, vmax=vmax, vmin=vmin)
axes[1].set_title("Electrodos - Post")

percent_change = (post_plcb - pre_plcb) / pre_plcb  # Convertir a %

sns.barplot(x=pre_plcb.columns, y=np.median(percent_change,axis=0)*100, ax=axes[2], color="steelblue")
axes[2].set_title("Cambio Mediano (%) por Electrodo")
axes[2].set_xlabel("Electrodos")
axes[2].set_ylabel("Cambio (%)")

plt.show()

fig, axes = plt.subplots(1, 3, figsize=(20, 5))

sns.heatmap(pre_exp, ax=axes[0], cmap="coolwarm", cbar=True, center=centro, vmax=vmax, vmin=vmin)
axes[0].set_title("Electrodos - Pre")

sns.heatmap(post_exp, ax=axes[1], cmap="coolwarm", cbar=True,center=centro, vmax=vmax, vmin=vmin)
axes[1].set_title("Electrodos - Post")
percent_change = (post_exp - pre_exp) / pre_exp  # Convertir a %

sns.barplot(x=pre_exp.columns, y=np.median(percent_change,axis=0)*100, ax=axes[2], color="steelblue")
axes[2].set_title("Cambio Mediano (%) por Electrodo")
axes[2].set_xlabel("Electrodos")
axes[2].set_ylabel("Cambio (%)")

plt.show()

# # Calculamos la diferencia de apEn entre POST-PRE
# diff_ctrl = Entropy.compute_diff(apEn_post_ctrl, apEn_pre_ctrl)

# diff_plcb = Entropy.compute_diff(apEn_post_plcb, apEn_pre_plcb)

# diff_exp = Entropy.compute_diff(apEn_post_exp, apEn_pre_exp)

# # Calculamos la diferencia mediana por electrodo en cada grupo

# # Approximation entropy: [ctrl_array, plcb_array, exp_array]
# apEn_arrays = [diff_ctrl.median(), diff_plcb.median(), diff_exp.median()]

# # Ploteamos diferencias
# Entropy.plot_diff(apEn_arrays, electrodes, title = "Target-Theta", subtitles = ["CTRL","PLCB","EXP"])

# # Corremos estadisticos 
# dict_apEn = {'CTRL': [apEn_pre_ctrl, apEn_post_ctrl],
#              'PLCB': [apEn_pre_plcb, apEn_post_plcb],
#              'EXP': [apEn_pre_exp, apEn_post_exp]} 

# pre = dict(zip(grupos, Utils.group_by(apEn_pre, col="grupo", val=grupos)))
# post = dict(zip(grupos, Utils.group_by(apEn_post, col="grupo", val=grupos)))   

# # Guardamos los resultados de los test pareados
# stats_results = Entropy.compute_paired_test(pre, post, channels, "wilcoxon")

# stats_results[(stats_results['p_value']<0.05)]

In [ ]:
# comp = 'all'

# CTRL = {'PRE': Entropy.combine_channels(apEn_pre_ctrl, eeg_regions).values(),
#         'POST': Entropy.combine_channels(apEn_post_ctrl, eeg_regions).values()}

# PLCB = {'PRE': Entropy.combine_channels(apEn_pre_plcb, eeg_regions).values(),
#        'POST': Entropy.combine_channels(apEn_post_plcb, eeg_regions).values()}

# EXP = {'PRE': Entropy.combine_channels(apEn_pre_exp, eeg_regions).values(),
#        'POST': Entropy.combine_channels(apEn_post_exp, eeg_regions).values()}

# Entropy.plot_by_regions(CTRL, PLCB, EXP, eeg_regions.keys())

Al comparar los instantes PRE y POST dentro de los grupos **PLCB** y **EXP**, presentan respuestas más **caóticas e impredecibles** en la banda **Theta**. En cambio, el grupo **CTRL** se mantiene constante.
- PLCB > EXP > CTRL

#### **Alpha**

In [ ]:
# Cargamos los datos en un DataFrame
df_alpha = pd.read_csv("./cerebral_bands/apEn_tgt_alpha.csv", header=0, delimiter=';')

# Seleccionamos la entropia de aproximacion por instante
apEn_pre, apEn_post = Utils.group_by(df_alpha[df_alpha['comp']=='all'], col="instante", val=["PRE","POST"])

# Seleccionamos la entropia de aproximacion por instante y de todo el estimulo (all).
apEn_pre_ctrl, apEn_pre_plcb, apEn_pre_exp = Utils.group_by(apEn_pre, col="grupo", val=grupos)
apEn_post_ctrl, apEn_post_plcb, apEn_post_exp = Utils.group_by(apEn_post, col="grupo", val=grupos)

# Calculamos la diferencia de apEn entre POST-PRE
diff_ctrl = Entropy.compute_diff(apEn_post_ctrl, apEn_pre_ctrl)

diff_plcb = Entropy.compute_diff(apEn_post_plcb, apEn_pre_plcb)

diff_exp = Entropy.compute_diff(apEn_post_exp, apEn_pre_exp)

# Calculamos la diferencia mediana por electrodo en cada grupo

# Approximation entropy: [ctrl_array, plcb_array, exp_array]
apEn_arrays = [diff_ctrl.median(), diff_plcb.median(), diff_exp.median()]

# Ploteamos diferencias
Entropy.plot_diff(apEn_arrays, electrodes, title = "Target-Alpha", subtitles = ["CTRL","PLCB","EXP"])

# Corremos estadisticos 
dict_apEn = {'CTRL': [apEn_pre_ctrl, apEn_post_ctrl],
             'PLCB': [apEn_pre_plcb, apEn_post_plcb],
             'EXP': [apEn_pre_exp, apEn_post_exp]} 

pre = dict(zip(grupos, Utils.group_by(apEn_pre, col="grupo", val=grupos)))
post = dict(zip(grupos, Utils.group_by(apEn_post, col="grupo", val=grupos)))   

# Guardamos los resultados de los test pareados
stats_results = Entropy.compute_paired_test(pre, post, channels, "wilcoxon")

stats_results[(stats_results['p_value']<0.05)]

In [ ]:
# comp = 'all'

# CTRL = {'PRE': Entropy.combine_channels(apEn_pre_ctrl, eeg_regions).values(),
#         'POST': Entropy.combine_channels(apEn_post_ctrl, eeg_regions).values()}

# PLCB = {'PRE': Entropy.combine_channels(apEn_pre_plcb, eeg_regions).values(),
#        'POST': Entropy.combine_channels(apEn_post_plcb, eeg_regions).values()}

# EXP = {'PRE': Entropy.combine_channels(apEn_pre_exp, eeg_regions).values(),
#        'POST': Entropy.combine_channels(apEn_post_exp, eeg_regions).values()}

# Entropy.plot_by_regions(CTRL, PLCB, EXP, eeg_regions.keys())

Los 3 grupos se mantienen en constantes en la banda **Alpha**

#### **Beta**

In [ ]:
# Cargamos los datos en un DataFrame
df_beta = pd.read_csv("./cerebral_bands/apEn_tgt_beta.csv", header=0, delimiter=';')

# Seleccionamos la entropia de aproximacion por instante
apEn_pre, apEn_post = Utils.group_by(df_beta[df_beta['comp']=='all'], col="instante", val=["PRE","POST"])

# Seleccionamos la entropia de aproximacion por instante y de todo el estimulo (all).
apEn_pre_ctrl, apEn_pre_plcb, apEn_pre_exp = Utils.group_by(apEn_pre, col="grupo", val=grupos)
apEn_post_ctrl, apEn_post_plcb, apEn_post_exp = Utils.group_by(apEn_post, col="grupo", val=grupos)

# Calculamos la diferencia de apEn entre POST-PRE
diff_ctrl = Entropy.compute_diff(apEn_post_ctrl, apEn_pre_ctrl)

diff_plcb = Entropy.compute_diff(apEn_post_plcb, apEn_pre_plcb)

diff_exp = Entropy.compute_diff(apEn_post_exp, apEn_pre_exp)

# Calculamos la diferencia mediana por electrodo en cada grupo

# Approximation entropy: [ctrl_array, plcb_array, exp_array]
apEn_arrays = [diff_ctrl.median(), diff_plcb.median(), diff_exp.median()]

# Ploteamos diferencias
Entropy.plot_diff(apEn_arrays, electrodes, title = "Target-Beta", subtitles = ["CTRL","PLCB","EXP"])

# Corremos estadisticos 
dict_apEn = {'CTRL': [apEn_pre_ctrl, apEn_post_ctrl],
             'PLCB': [apEn_pre_plcb, apEn_post_plcb],
             'EXP': [apEn_pre_exp, apEn_post_exp]} 

pre = dict(zip(grupos, Utils.group_by(apEn_pre, col="grupo", val=grupos)))
post = dict(zip(grupos, Utils.group_by(apEn_post, col="grupo", val=grupos)))   

# Guardamos los resultados de los test pareados
stats_results = Entropy.compute_paired_test(pre, post, eeg_regions, "wilcoxon")

stats_results[(stats_results['p_value']<0.05)]

In [ ]:
# comp = 'all'

# CTRL = {'PRE': Entropy.combine_channels(apEn_pre_ctrl, eeg_regions).values(),
#         'POST': Entropy.combine_channels(apEn_post_ctrl, eeg_regions).values()}

# PLCB = {'PRE': Entropy.combine_channels(apEn_pre_plcb, eeg_regions).values(),
#        'POST': Entropy.combine_channels(apEn_post_plcb, eeg_regions).values()}

# EXP = {'PRE': Entropy.combine_channels(apEn_pre_exp, eeg_regions).values(),
#        'POST': Entropy.combine_channels(apEn_post_exp, eeg_regions).values()}

# Entropy.plot_by_regions(CTRL, PLCB, EXP, eeg_regions.keys())

Al comparar los instantes PRE y POST dentro de los grupos **CTRL** y **EXP**, presentan respuestas más caóticas e impredecibles en la banda **Beta**. En cambio, el grupo **PLCB** se mantiene constante.
- CTRL > EXP > PLCB

-------

## Potencia espectral

La potencia espectral cruzada es una medida que nos dice cómo están relacionadas dos señales mediante la energía que comparten en cada frecuencia. 

$$P_{XY}(f) = X(f)·Y^{*}(f)$$ 

- Donde:
    - $X(f)$: son los componentes de la DFT correspondientes a $x[n]$.
    - $Y(f)$: son los componentes conjugados de la DFT correspondientes a $y[n]$. 

Su demostración es posible mediante las propiedades de los números complejos en coordenadas polares, ya que cada componente de la DFT puede ser transformada de forma directa.

- En coordenadas polares, se representa como:

$$X(f) = |X(f)|·e^{i\phi}$$

Donde:
- $|X(f)|$: es la magnitud.
- $\phi$: es la fase.

La DFT toma una señal de $N$ puntos y devuelve $N$ valores complejos en el dominio de la frecuencia. Estos valores corresponden a $N$ frecuencias equiespaciadas, que van desde 0 Hz hasta $F_{s}$ Hz (frecuencia de muestreo). Sin embargo, no todas las frecuencias son únicas, ya que la distribución de sus valores es simétrica y solo nos interesan aquellas que van desde 0 Hz hasta $\frac{F_{s}}{2}$ Hz (frecuencia de Nyquist).

Esas componentes de frecuencia tienen la forma $Re + j·Im$ y pueden transformarse perfectamente en coordenadas polares $X(f) = |X(f)|·e^{i\phi}$. 

La potencia espectral cruzada trata de combinar la información que refleja la magnitud y fase asociadas a cada frecuencia entre dos señales. 

En términos de magnitud, si multiplicamos la magnitud por cada par de frecuencias en ambas señales, los valores grandes se acentuarán y los demás se mantendrán en las mismas proporciones. Esto información permite resaltar de forma sencilla las frecuencias que poseen una magnitud significativa común.

$$|X(f)|·|Y(f)|$$

En términos de fase, la diferencia por cada par de frecuencias en ambas señales permite conocer su desfase o sincronización.

$$\phi_{X}-\phi_{Y}$$

A través de las propiedades de las potencias en coordenadas polares, la combinación de la magnitud y desfase es automática como se puede observar, a continuación:

- Ejemplo:
$$z_{x}=|z_{x}|·e^{i(\phi_{x})}\ y\ z_{y}=|z_{y}|·e^{i(\phi_{y})}$$

$$z_{x}·z^{*}_{y} = |z_{x}|·|z_{y}|·e^{i(\phi_{x}-\phi_{y})}$$

- Fórmula:

$$P_{xy}(f)=X(f)·Y^{*}(f)=|X(f)|·|Y(f)|·e^{i(\phi_{X}-\phi_{Y})}$$

## Spectral coherence

La coherencia espectral es una medida que nos dice cuánto se parecen dos señales en cada frecuencia. Se puede entender como:

- La proporción de la potencia de $y[n]$ que se explica por $x[n]$ en cada frecuencia $f$.
- Normalización de la potencia espectral cruzada entre dos señales.


$$C_{XY}(f) = \frac{|P_{XY}(f)|^{2}}{P_{XX}(f)·P_{YY}(f)}$$

- Donde:
    - $P_{XY}(f)$: es la potencia espectral cruzada entre las señales $x[n]$ e $y[n]$.
    - $P_{XX}(f)$: es la densidad espectral de potencia de la señal $x[n]$.
    - $P_{YY}(f)$: es la densidad espectral de potencia de la señal $y[n]$.
    - $C_{XY}(f)$: es la coherencia espectral que toma valores entre 0 y 1.



La coherencia espectral mide la relación en el dominio de la frecuencia entre dos señales. Para hacerlo, la función *signal.coherence()*:

- Divide la señal en segmentos (nperseg) de longitud determinada.

- Aplica una FFT a cada segmento para obtener las frecuencias de cada segmento.

- Luego, calcula las densidades espectrales y las densidades espectrales cruzadas para calcular la coherencia a través de un rango de frecuencias.

La FFT realiza una transformada espectral que devuelve un rango de frecuencias, desde 0 Hz hasta Nyquist (la mitad de la frecuencia de muestreo, fs/2). La coherencia no solo mide la frecuencia de las señales, sino todas las frecuencias en las cuales hay componentes en las señales.


In [ ]:
# Paths with files (.csv)
path_tgt = "E:/TFM/CLUSTERING_ALL_ICA_by_segments/ODDBALL/stimulus_tgt.csv"
path_std = "E:/TFM/CLUSTERING_ALL_ICA_by_segments/ODDBALL/stimulus_std.csv"

# Load DataFrame for every type of stimulus
df_w_tgt = pd.read_csv(path_tgt, header=0, delimiter=';')
df_w_std = pd.read_csv(path_std, header=0, delimiter=';')

# Common information about both DataFrames, where: df_std.columns == df_tgt.columns
channels = df_w_std.columns[6:].to_list()
info = df_w_std.columns[:6].to_list()
components = df_w_std["comp"].unique()

# Seleccionamos instantes y grupos de TARGET DataFrames
df_tgt_ctrl = df_w_tgt[df_w_tgt["grupo"] == "CONTROL"]# select CTRL subjects
df_tgt_plcb = df_w_tgt[df_w_tgt["grupo"] == "PLCB"]# select PLCB subjects
df_tgt_exp = df_w_tgt[df_w_tgt["grupo"] == "EXP"]# select EXP subjects

df_tgt_pre_ctrl = df_w_tgt[(df_w_tgt["instante"]=="PRE")&(df_w_tgt["grupo"]=="CONTROL")]
df_tgt_pre_plcb = df_w_tgt[(df_w_tgt["instante"]=="PRE")&(df_w_tgt["grupo"]=="PLCB")]
df_tgt_pre_exp = df_w_tgt[(df_w_tgt["instante"]=="PRE")&(df_w_tgt["grupo"]=="EXP")]

df_tgt_post_ctrl = df_w_tgt[(df_w_tgt["instante"]=="POST")&(df_w_tgt["grupo"]=="CONTROL")]
df_tgt_post_plcb = df_w_tgt[(df_w_tgt["instante"]=="POST")&(df_w_tgt["grupo"]=="PLCB")]
df_tgt_post_exp = df_w_tgt[(df_w_tgt["instante"]=="POST")&(df_w_tgt["grupo"]=="EXP")]

df_tgt_seg_ctrl = df_w_tgt[(df_w_tgt["instante"]=="SEGUIMIENTO")&(df_w_tgt["grupo"]=="CONTROL")]
df_tgt_seg_plcb = df_w_tgt[(df_w_tgt["instante"]=="SEGUIMIENTO")&(df_w_tgt["grupo"]=="PLCB")]
df_tgt_seg_exp = df_w_tgt[(df_w_tgt["instante"]=="SEGUIMIENTO")&(df_w_tgt["grupo"]=="EXP")]

# Seleccionamos sujetos comunes entre instantes
df_tgt_pre_ctrl,df_tgt_post_ctrl = Utils.select_common_ids(df_tgt_pre_ctrl, df_tgt_post_ctrl)
df_tgt_pre_plcb,df_tgt_post_plcb = Utils.select_common_ids(df_tgt_pre_plcb, df_tgt_post_plcb)
df_tgt_pre_exp,df_tgt_post_exp = Utils.select_common_ids(df_tgt_pre_exp, df_tgt_post_exp)

print("Header with info:",info)
print("Channels:",channels)
print("Components:",components)

In [ ]:
# Definir las bandas cerebrales (en Hz)
dict_bands = {
    'all': (1, 30),
    'delta': (1, 4),
    'theta': (4, 8),
    'alpha': (8, 12),
    'beta': (12, 30)}

diff_tgt_coherence = {}

for band, freqs in dict_bands.items():

    # Compute coherence matrix for every subject between every pair of channels
    coherence_pre_ctrl = Coherence.compute_matrix(df_tgt_pre_ctrl, channels, 'all', fs=500, nperseg=225, band_range=freqs)
    coherence_post_ctrl = Coherence.compute_matrix(df_tgt_post_ctrl, channels, 'all', fs=500, nperseg=225, band_range=freqs)

    # Check if subjects are correctly ordered and each pair has the same number of values
    Coherence.check_consistency(coherence_pre_ctrl, coherence_post_ctrl)

    # Compute coherence matrix for every subject between every pair of channels
    coherence_pre_plcb = Coherence.compute_matrix(df_tgt_pre_plcb, channels, 'all', fs=500, nperseg=225, band_range=freqs)
    coherence_post_plcb = Coherence.compute_matrix(df_tgt_post_plcb, channels, 'all', fs=500, nperseg=225, band_range=freqs)

    # Check if subjects are correctly ordered and each pair has the same number of values
    Coherence.check_consistency(coherence_pre_plcb, coherence_post_plcb)

    # Compute coherence matrix for every subject between every pair of channels
    coherence_pre_exp = Coherence.compute_matrix(df_tgt_pre_exp, channels, 'all', fs=500, nperseg=225, band_range=freqs)
    coherence_post_exp = Coherence.compute_matrix(df_tgt_post_exp, channels, 'all', fs=500, nperseg=225, band_range=freqs)

    # Check if subjects are correctly ordered and each pair has the same number of values 
    Coherence.check_consistency(coherence_pre_exp, coherence_post_exp)

    # Compute POST - PRE differences between each pair of coherence matrices
    diff_ctrl = Coherence.get_diff(coherence_post_ctrl, coherence_pre_ctrl)
    diff_plcb = Coherence.get_diff(coherence_post_plcb, coherence_pre_plcb)
    diff_exp = Coherence.get_diff(coherence_post_exp, coherence_pre_exp)

    # Calculamos la matriz mediana de todos los controles, placebos y experimentales
    diff_tgt_coherence[band] = [np.median(diff_ctrl, axis=0), np.median(diff_plcb, axis=0), np.median(diff_exp, axis=0)]

In [ ]:
Coherence.plot_diff(diff_tgt_coherence["all"], channels, titles = ["Target", "All", "CTRL","PLCB","EXP"])

In [ ]:
Coherence.plot_diff(diff_tgt_coherence["delta"], channels, titles = ["Target", "Delta", "CTRL","PLCB","EXP"])

In [ ]:
Coherence.plot_diff(diff_tgt_coherence["theta"], channels, titles = ["Target", "Theta", "CTRL","PLCB","EXP"])

In [ ]:
Coherence.plot_diff(diff_tgt_coherence["alpha"], channels, titles = ["Target", "Alpha", "CTRL","PLCB","EXP"])

In [ ]:
Coherence.plot_diff(diff_tgt_coherence["beta"], channels, titles = ["Target", "Beta", "CTRL","PLCB","EXP"])

## Neural response

In [ ]:
def plot_graph(sync_matrix, channels, electrodes, title="TITLE", subtitle=["CTRL","PLCB","EXP"]):

    circunferencias = np.arange(0, 1.5, 0.2)
    vmin = min(matrix.min() for matrix in sync_matrix)  # Encuentra el minimo global
    vmax = max(matrix.max() for matrix in sync_matrix)  # Encuentra el maximo global
    vmax = max(abs(vmin), abs(vmax))
    vmin = -vmax
    cut_off = np.median(np.abs(sync_matrix))

    # Dibujar los nodos
    x_coords, y_coords = zip(*electrodes.values())

    # Crear una figura con 1 fila y 4 columnas
    fig, axes = plt.subplots(1, 4, figsize=(18, 6), gridspec_kw={"width_ratios": [1, 1, 1, 0.05]})

    # Colocamos Titulo de la figura
    plt.suptitle('['+title+']'+r' Neural Response Graph ($\Delta$ POST-PRE)', fontsize=16)

    # Por cada eje o subplot
    for k, ax in enumerate(axes[:-1]):

        # Dibujamos aristas como nivel de coherencia
        for i in range(len(channels)):
            for j in range(i + 1, len(channels)):  # Evitar duplicados

                weight = sync_matrix[k][i, j]# mismo orden en columnas y filas que lista 'channels'

                if abs(weight) > cut_off:  # Filtrar conexiones debiles
                    # Nos aseguramos de seleccionar correctamente los canales en orden
                    x1, y1 = electrodes[channels[i]]
                    x2, y2 = electrodes[channels[j]]

                    normalized_weight = (weight - (-0.2)) / (0.2 - (-0.2))# Escala entre 0 y 1

                    color = plt.cm.seismic(normalized_weight)# Necesita valores entre 0 y 1 para adjudicar colores correctamente

                    linewidth = abs(weight)*15  # Grosor proporcional
                    # Graficamos
                    ax.plot([x1, x2], [y1, y2], color=color, linewidth=linewidth, alpha=0.8)

        # Fondo gris
        ax.set_facecolor('lightgray')

        ax.set_title(subtitle[k])
        ax.scatter(x_coords, y_coords, c="gray", s=200, edgecolors="black", zorder=3)
        ax.set_xticks([])  # Eliminar las marcas en el eje X
        ax.set_yticks([])  # Eliminar las marcas en el eje Y
        ax.set_xlim([-1.2, 1.2])
        ax.set_ylim([-1.0, 1.0])

        # Agregar etiquetas con alineacion condicional
        for label, (x, y) in electrodes.items():
            ha = "left" if x > 0 else "right" if x < 0 else "center"
            ax.text(x, y+0.05, label, fontsize=12, ha=ha, va='bottom')

        # Dibujar las circunferencias en el gráfico
        for r in circunferencias:
            circle = plt.Circle((0,0), r, color='gray', fill=False, linestyle='--', linewidth=0.5)
            ax.add_artist(circle)

    # Ocultamos los ejes de la cuarta y quinta figura
    axes[-1].axis('off')

    # Crear la barra de colores en la cuarta columna 
    # Necesitamos crear una imagen ficticia para la colorbar. Usamos una dispersión
    sc = axes[0].scatter([], [], c=[], cmap="bwr", vmin=vmin, vmax=vmax)  # Definimos un scatter vacío con la colormap y límites

    cbar = fig.colorbar(sc, ax=axes, orientation='vertical', fraction=0.02, pad=0.01)
    cbar.set_label('Increase (red) to Decrease (blue)', fontsize=12, rotation=270, labelpad=20)

    cbar.set_ticks([])

    # Ajustamos diseño manualmente
    fig.subplots_adjust(left=0.05, right=0.9, wspace=0.15)
    plt.show()
    return

In [ ]:
# Seleccionamos la entropia de aproximacion por instante
apEn_pre, apEn_post = Utils.group_by(df_tgt_apEn[df_tgt_apEn['comp']=='all'], col="instante", val=["PRE","POST"])

# Seleccionamos la entropia de aproximacion por instante y de todo el estimulo (all).
apEn_pre_ctrl, apEn_pre_plcb, apEn_pre_exp = Utils.group_by(apEn_pre, col="grupo", val=grupos)
apEn_post_ctrl, apEn_post_plcb, apEn_post_exp = Utils.group_by(apEn_post, col="grupo", val=grupos)

# Calculamos la diferencia de apEn entre POST-PRE
diff_ctrl = Entropy.compute_diff(apEn_post_ctrl, apEn_pre_ctrl)

diff_plcb = Entropy.compute_diff(apEn_post_plcb, apEn_pre_plcb)

diff_exp = Entropy.compute_diff(apEn_post_exp, apEn_pre_exp)

# Calculamos la diferencia mediana por electrodo en cada grupo

# Approximation entropy: [ctrl_array, plcb_array, exp_array]
apEn_arrays = [diff_ctrl.median(), diff_plcb.median(), diff_exp.median()]

# Coherence matrix: [ctrl_matrix, plcb_matrix, exp_matrix]
coherence_matrices = diff_tgt_coherence["all"]



plot_graph(coherence_matrices, channels, electrodes, title="Target-All", subtitle=["CTRL","PLCB","EXP"])

In [ ]:
# Seleccionamos la entropia de aproximacion por instante
apEn_pre, apEn_post = Utils.group_by(df_delta, col="instante", val=["PRE","POST"])

# Seleccionamos la entropia de aproximacion por instante y de todo el estimulo (all).
apEn_pre_ctrl, apEn_pre_plcb, apEn_pre_exp = Utils.group_by(apEn_pre[apEn_pre['comp']=='all'], col="grupo", val=["CONTROL", "PLCB", "EXP"])
apEn_post_ctrl, apEn_post_plcb, apEn_post_exp = Utils.group_by(apEn_post[apEn_post['comp']=='all'], col="grupo", val=["CONTROL", "PLCB", "EXP"])

# Calculamos la diferencia de apEn entre POST-PRE
diff_ctrl = Entropy.compute_diff(apEn_post_ctrl[apEn_post_ctrl['comp']=='all'], 
                                 apEn_pre_ctrl[apEn_pre_ctrl['comp']=='all'])

diff_plcb = Entropy.compute_diff(apEn_post_plcb[apEn_post_plcb['comp']=='all'], 
                                 apEn_pre_plcb[apEn_pre_plcb['comp']=='all'])

diff_exp = Entropy.compute_diff(apEn_post_exp[apEn_post_exp['comp']=='all'], 
                                apEn_pre_exp[apEn_pre_exp['comp']=='all'])

# Calculamos la diferencia mediana por electrodo en cada grupo

# Approximation entropy: [ctrl_array, plcb_array, exp_array]
apEn_arrays = [diff_ctrl.median(), diff_plcb.median(), diff_exp.median()]

# Coherence matrix: [ctrl_matrix, plcb_matrix, exp_matrix]
coherence_matrices = diff_tgt_coherence["delta"]

plot_graph(coherence_matrices, channels, electrodes, title="Target-Delta", subtitle=["CTRL","PLCB","EXP"])

In [ ]:
# Seleccionamos la entropia de aproximacion por instante
apEn_pre, apEn_post = Utils.group_by(df_theta, col="instante", val=["PRE","POST"])

# Seleccionamos la entropia de aproximacion por instante y de todo el estimulo (all).
apEn_pre_ctrl, apEn_pre_plcb, apEn_pre_exp = Utils.group_by(apEn_pre[apEn_pre['comp']=='all'], col="grupo", val=["CONTROL", "PLCB", "EXP"])
apEn_post_ctrl, apEn_post_plcb, apEn_post_exp = Utils.group_by(apEn_post[apEn_post['comp']=='all'], col="grupo", val=["CONTROL", "PLCB", "EXP"])

# Calculamos la diferencia de apEn entre POST-PRE
diff_ctrl = Entropy.compute_diff(apEn_post_ctrl[apEn_post_ctrl['comp']=='all'], 
                                 apEn_pre_ctrl[apEn_pre_ctrl['comp']=='all'])

diff_plcb = Entropy.compute_diff(apEn_post_plcb[apEn_post_plcb['comp']=='all'], 
                                 apEn_pre_plcb[apEn_pre_plcb['comp']=='all'])

diff_exp = Entropy.compute_diff(apEn_post_exp[apEn_post_exp['comp']=='all'], 
                                apEn_pre_exp[apEn_pre_exp['comp']=='all'])

# Approximation entropy: [ctrl_array, plcb_array, exp_array]
apEn_arrays = [diff_ctrl.median(), diff_plcb.median(), diff_exp.median()]

# Coherence matrix: [ctrl_matrix, plcb_matrix, exp_matrix]
coherence_matrices = diff_tgt_coherence["theta"]

plot_graph(coherence_matrices, channels, electrodes, title="Target-Theta", subtitle=["CTRL","PLCB","EXP"])

In [ ]:
# Seleccionamos la entropia de aproximacion por instante
apEn_pre, apEn_post = Utils.group_by(df_alpha, col="instante", val=["PRE","POST"])

# Seleccionamos la entropia de aproximacion por instante y de todo el estimulo (all).
apEn_pre_ctrl, apEn_pre_plcb, apEn_pre_exp = Utils.group_by(apEn_pre[apEn_pre['comp']=='all'], col="grupo", val=["CONTROL", "PLCB", "EXP"])
apEn_post_ctrl, apEn_post_plcb, apEn_post_exp = Utils.group_by(apEn_post[apEn_post['comp']=='all'], col="grupo", val=["CONTROL", "PLCB", "EXP"])

# Calculamos la diferencia de apEn entre POST-PRE
diff_ctrl = Entropy.compute_diff(apEn_post_ctrl[apEn_post_ctrl['comp']=='all'], 
                                 apEn_pre_ctrl[apEn_pre_ctrl['comp']=='all'])

diff_plcb = Entropy.compute_diff(apEn_post_plcb[apEn_post_plcb['comp']=='all'], 
                                 apEn_pre_plcb[apEn_pre_plcb['comp']=='all'])

diff_exp = Entropy.compute_diff(apEn_post_exp[apEn_post_exp['comp']=='all'], 
                                apEn_pre_exp[apEn_pre_exp['comp']=='all'])

# Approximation entropy: [ctrl_array, plcb_array, exp_array]
apEn_arrays = [diff_ctrl.median(), diff_plcb.median(), diff_exp.median()]

# Coherence matrix: [ctrl_matrix, plcb_matrix, exp_matrix]
coherence_matrices = diff_tgt_coherence["alpha"]

plot_graph(coherence_matrices, channels, electrodes, title="Target-Alpha", subtitle=["CTRL","PLCB","EXP"])

In [ ]:
# Seleccionamos la entropia de aproximacion por instante
apEn_pre, apEn_post = Utils.group_by(df_beta, col="instante", val=["PRE","POST"])

# Seleccionamos la entropia de aproximacion por instante y de todo el estimulo (all).
apEn_pre_ctrl, apEn_pre_plcb, apEn_pre_exp = Utils.group_by(apEn_pre[apEn_pre['comp']=='all'], col="grupo", val=["CONTROL", "PLCB", "EXP"])
apEn_post_ctrl, apEn_post_plcb, apEn_post_exp = Utils.group_by(apEn_post[apEn_post['comp']=='all'], col="grupo", val=["CONTROL", "PLCB", "EXP"])

# Calculamos la diferencia de apEn entre POST-PRE
diff_ctrl = Entropy.compute_diff(apEn_post_ctrl[apEn_post_ctrl['comp']=='all'], 
                                 apEn_pre_ctrl[apEn_pre_ctrl['comp']=='all'])

diff_plcb = Entropy.compute_diff(apEn_post_plcb[apEn_post_plcb['comp']=='all'], 
                                 apEn_pre_plcb[apEn_pre_plcb['comp']=='all'])

diff_exp = Entropy.compute_diff(apEn_post_exp[apEn_post_exp['comp']=='all'], 
                                apEn_pre_exp[apEn_pre_exp['comp']=='all'])

# Approximation entropy: [ctrl_array, plcb_array, exp_array]
apEn_arrays = [diff_ctrl.median(), diff_plcb.median(), diff_exp.median()]

# Coherence matrix: [ctrl_matrix, plcb_matrix, exp_matrix]
coherence_matrices = diff_tgt_coherence["beta"]

plot_graph(coherence_matrices, channels, electrodes, title="Target-Beta", subtitle=["CTRL","PLCB","EXP"])

# Time response

$$Event\ Related\ Potentials\ (ERP)\ Components$$

$$
\begin{array}{|c|c|c|}
\hline
\textbf{Componentes} & \textbf{$t_{o}$ [ms]} & \textbf{$t_{f}$ [ms]} \\
\hline
\text{P1} & 80 & 130 \\ \hline
\text{N1} & 130 & 200 \\ \hline
\text{P2} & 200 & 300 \\ \hline
\text{N2} & 300 & 360 \\ \hline
\text{P3} & 360 & 600 \\ \hline
\end{array}
$$

Se han llevado a cabo dos tipos de contrastes:

1) Para cada instante, se han analizado los grupos como distribuciones independientes.

    - Instante PRE:
        - CTRL vs PLCB
        - CTRL vs EXP
        - EXP vs PLCB

    - Instante POST:
        - CTRL vs PLCB
        - CTRL vs EXP
        - EXP vs PLCB

    - Conclusión: 

        - En el instante PRE, los grupos presentan diferencias significativas entre sí, lo cual resulta inconsistente, ya que se asume que parten de condiciones similares. 
        - A la luz de estos resultados y los obtenidos en el instante POST, aunque también muestren diferencias significativas, no parecen ser del todo fiables. 
        - Descarto este tipo de comparación y solo tengo en cuenta los contrastes de instantes dentro de cada grupo.

2) Para cada grupo, se ha analizado el cambio entre los instantes PRE y POST como distribuciones pareadas.

    - Grupo CTRL:
        - PRE vs POST
    - Grupo PLCB:
        - PRE vs POST
    - Grupo EXP:
        - PRE vs POST

In [ ]:
# Paths with files (.csv)
path_tgt = "E:/TFM/CLUSTERING_ALL_ICA_by_segments/ODDBALL/times_tgt.csv"
path_std = "E:/TFM/CLUSTERING_ALL_ICA_by_segments/ODDBALL/times_std.csv"

# Load DataFrame for every type of stimulus
df_t_tgt, df_t_std = pd.read_csv(path_tgt, header=0, delimiter=';'), pd.read_csv(path_std, header=0, delimiter=';')

# Common information about both DataFrames, where: df_std.columns == df_tgt.columns
channels = df_t_tgt.columns[6:].to_list()
info = df_t_tgt.columns[:6].to_list()
components = df_t_tgt["comp"].unique()

# Seleccionamos instantes de TARGET DataFrames
tgt_instante = dict(zip(["PRE","POST"], Utils.group_by(df_t_tgt, col="instante", val=["PRE", "POST"])))
# Seleccionamos instantes de STANDARD DataFrames
std_instante = dict(zip(["PRE","POST"], Utils.group_by(df_t_std, col="instante", val=["PRE", "POST"])))

# Seleccionamos instante PRE de TARGET DataFrames
tgt_pre_grupo = dict(zip(["CONTROL","PLCB","EXP"], Utils.group_by(tgt_instante["PRE"], col="grupo", val=["CONTROL", "PLCB", "EXP"])))
# Seleccionamos instante PRE de STANDARD DataFrames
std_pre_grupo = dict(zip(["CONTROL","PLCB","EXP"], Utils.group_by(std_instante["PRE"], col="grupo", val=["CONTROL", "PLCB", "EXP"])))

# Seleccionamos instante POST de TARGET DataFrames
tgt_post_grupo = dict(zip(["CONTROL","PLCB","EXP"], Utils.group_by(tgt_instante["POST"], col="grupo", val=["CONTROL", "PLCB", "EXP"])))
# Seleccionamos instante POST de STANDARD DataFrames
std_post_grupo = dict(zip(["CONTROL","PLCB","EXP"], Utils.group_by(std_instante["POST"], col="grupo", val=["CONTROL", "PLCB", "EXP"])))

# Seleccionamos sujetos comunes entre instantes
tgt_pre_grupo["CONTROL"], tgt_post_grupo["CONTROL"] = Utils.select_common_ids(tgt_pre_grupo["CONTROL"], tgt_post_grupo["CONTROL"])
tgt_pre_grupo["PLCB"], tgt_post_grupo["PLCB"] = Utils.select_common_ids(tgt_pre_grupo["PLCB"], tgt_post_grupo["PLCB"])
tgt_pre_grupo["EXP"], tgt_post_grupo["EXP"] = Utils.select_common_ids(tgt_pre_grupo["EXP"], tgt_post_grupo["EXP"])

std_pre_grupo["CONTROL"], std_post_grupo["CONTROL"] = Utils.select_common_ids(std_pre_grupo["CONTROL"], std_post_grupo["CONTROL"])
std_pre_grupo["PLCB"], std_post_grupo["PLCB"] = Utils.select_common_ids(std_pre_grupo["PLCB"], std_post_grupo["PLCB"])
std_pre_grupo["EXP"], std_post_grupo["EXP"] = Utils.select_common_ids(std_pre_grupo["EXP"], std_post_grupo["EXP"])

print("Header with info:",info)
print("Channels:",channels)
print("Components:",components)

best_channels = list(set(channels) ^ set(['P8','T8', 'P4', 'O1', 'Oz', 'O2', 'Pz', 'P3', 'T7', 'P7']))  # Operador XOR (^) en conjuntos

In [ ]:
def compute_diff(tgt_post, tgt_pre, groups, channels, c):

    data = []
    for g in groups:
        df1, df0 = tgt_post[g], tgt_pre[g]
        diff = []
        for (n,id), df0_id in df0.groupby(["n_test","id"]):

            df1_id = Utils.group_by(df1[df1['n_test']==n], col="id", val=[id])[0]

            post = df1_id[df1_id['comp']==c].loc[:,channels].values
            pre = df0_id[df0_id['comp']==c].loc[:,channels].values

            d = post-pre
            diff.append(d)
        data.append(np.vstack(diff))
    return data


In [ ]:

for c in components:
    ctrl, plcb, exp = compute_diff(tgt_post_grupo, tgt_pre_grupo, ["CONTROL", "PLCB", "EXP"], channels, c)
        

    # Circunferencias concentricas 
    circunferencias = np.arange(0, 1.5, 0.2)

    # Extraemos las coordenadas
    x = [coord[0] for coord in electrodes.values()]
    y = [coord[1] for coord in electrodes.values()]

    data = [np.median(ctrl, axis=0), np.median(plcb, axis = 0), np.median(exp, axis=0)]
    lim = np.max([np.abs(np.min(data)), np.abs(np.max(data))])

    import matplotlib.pyplot as plt
    title = c
    subtitles = ["CTRL","PLCB","EXP"]
    # Crear una figura con 1 fila y 3 columnas
    fig, axes = plt.subplots(1, 4, figsize=(18, 6), gridspec_kw={"width_ratios": [1, 1, 1, 0.05]})
    plt.suptitle('['+title+']'+r' Time ($\Delta$ POST-PRE)', fontsize=16)

    for i, ax in enumerate(axes[:-1]):
        for j in range(len(channels)):
            x, y = electrodes[channels[j]]
            sc = ax.scatter(x, y, c=data[i][j], cmap="coolwarm", s=200, vmin=-lim, vmax = lim)
        ax.set_facecolor('lightgray')
        ax.set_title(subtitles[i])
        ax.set_xticks([])  # Eliminar las marcas en el eje X
        ax.set_yticks([])  # Eliminar las marcas en el eje Y
        ax.set_xlim([-1.2, 1.2])
        ax.set_ylim([-1.0, 1.0])
        ax.grid(True)

        # Etiquetas de los electrodos en cada grafico
        for label, (i, j) in electrodes.items():
            ha = "left" if i > 0 else "right" if i < 0 else "center"
            ax.text(i, j, label, fontsize=14, ha=ha)

        # Dibujar las circunferencias en el gráfico
        for r in circunferencias:
            circle = plt.Circle((0,0), r, color='gray', fill=False, linestyle='--', linewidth=0.5)
            ax.add_artist(circle)

    # Ocultamos los ejes de la cuarta figura
    axes[-1].axis('off')

    # Crear la barra de colores en la cuarta columna
    fig.colorbar(sc, ax=axes, orientation='vertical', fraction=0.02, pad=0.04)

    # Ajustamos diseño manualmente
    fig.subplots_adjust(left=0.05, right=0.9, wspace=0.15)

    # Mostrar el grafico
    plt.show()

In [ ]:
import pandas as pd
from scipy.stats import ttest_rel
from statsmodels.stats.multitest import multipletests
import numpy as np

# Suponiendo que ya tienes tus datos y utilidades listos
eeg_regions = {
    "Frontal": ["Fp1", "Fp2", "F3", "Fz", "F4", "F7", "F8"],
    "Central": ["C3", "Cz", "C4"],
    "Parietal": ["P3", "Pz", "P4"],
    "Occipital": ["O1", "Oz", "O2"],
    "Temporal": ["T7", "T8"],
    "Parieto-temporal": ["P7", "P8"],
    "Parieto-occipital": ["P3", "P4"],
    "Linea-media": ["Fz", "Cz", "Pz"]  
}

# Lista vacía para almacenar los resultados
results = []

# Función para calcular la d de Cohen para muestras pareadas
def cohen_d_paired(post, pre):
    # Calcular las diferencias entre las mediciones pre y post
    differences = post - pre
    # Calcular la desviación estándar de las diferencias
    sd_diff = np.std(differences, ddof=1)
    # Calcular la media de las diferencias
    mean_diff = np.mean(differences)
    # Calcular d de Cohen
    d = mean_diff / sd_diff
    return d

# Iterar sobre las regiones, componentes y grupos
for k, v in eeg_regions.items():
    for c in components:  # Asumiendo que 'components' está definido
        for g in ["CONTROL", "PLCB", "EXP"]:
            
            # Filtrar datos para el grupo y componente
            ctrl_pre = Utils.group_by(tgt_pre_grupo[g], col="comp", val=[c])[0]
            ctrl_post = Utils.group_by(tgt_post_grupo[g], col="comp", val=[c])[0]

            # Calcular los promedios de las regiones por sujeto
            lista_pre = ctrl_pre.loc[:, v].mean(axis=1)
            lista_post = ctrl_post.loc[:, v].mean(axis=1)

            # Aplicar t-test pareado
            t_stat, p_value = ttest_rel(lista_post,lista_pre)

            # Calcular la d de Cohen
            cohen_d = cohen_d_paired(lista_post, lista_pre)

            # Determinar si PRE > POST o PRE < POST
            if t_stat > 0:
                pre_vs_post = "PRE < POST"
            elif t_stat < 0:
                pre_vs_post = "PRE > POST"
            else:
                pre_vs_post = "PRE = POST"

            # Guardar los resultados si p-value es significativo
            if p_value < 0.05:
                result = {
                    'Region': k,
                    'Component': c,
                    'Group': g,
                    't-statistic': t_stat,
                    'p-value': p_value,
                    'Cohen-d': cohen_d,
                    'PRE vs POST': pre_vs_post
                }

                # Agregar el resultado al listado
                results.append(result)

# Convertir el listado de resultados en un DataFrame
results_df = pd.DataFrame(results)

# Ajustar los p-valores usando el método de Benjamini-Hochberg (FDR)
results_df['p-value_adjusted'] = multipletests(results_df['p-value'], method='fdr_bh')[1]

# Mostrar el DataFrame con los resultados ajustados
results_df


In [ ]:
ctrl_pre = tgt_pre_grupo["CONTROL"]
ctrl_post = tgt_post_grupo["CONTROL"]

data_pre = [ctrl_pre[ctrl_pre['comp']==c].loc[:, ['Fz','Pz','Cz']].values.flatten() for c in components]
data_post = [ctrl_post[ctrl_post['comp']==c].loc[:, ['Fz','Pz','Cz']].values.flatten() for c in components]

import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

# Crear la figura con subgráficos
fig, axes = plt.subplots(1, len(components), figsize=(15, 5))

# Graficar cada par Pre/Post en un subplot
for i, c in enumerate(components):
    df = pd.DataFrame({
        "Valores": np.concatenate([data_pre[i], data_post[i]]),
        "Condición": ["Pre"] * len(data_pre[i]) + ["Post"] * len(data_post[i])
    })
    
    sns.boxplot(x="Condición", y="Valores", data=df, ax=axes[i])
    axes[i].set_title(f"Comparación para {c}")
    axes[i].set_xlabel("Condición")
    axes[i].set_ylabel("Valores")

# Ajustar el diseño y mostrar
plt.tight_layout()
plt.show()

In [ ]:
ctrl_pre = tgt_pre_grupo["PLCB"]
ctrl_post = tgt_post_grupo["PLCB"]

data_pre = [ctrl_pre[ctrl_pre['comp']==c].loc[:, channels].values.flatten() for c in components]
data_post = [ctrl_post[ctrl_post['comp']==c].loc[:, channels].values.flatten() for c in components]

import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

# Crear la figura con subgráficos
fig, axes = plt.subplots(1, len(components), figsize=(15, 5))

# Graficar cada par Pre/Post en un subplot
for i, c in enumerate(components):
    df = pd.DataFrame({
        "Valores": np.concatenate([data_pre[i], data_post[i]]),
        "Condición": ["Pre"] * len(data_pre[i]) + ["Post"] * len(data_post[i])
    })
    
    sns.boxplot(x="Condición", y="Valores", data=df, ax=axes[i])
    axes[i].set_title(f"Comparación para {c}")
    axes[i].set_xlabel("Condición")
    axes[i].set_ylabel("Valores")

# Ajustar el diseño y mostrar
plt.tight_layout()
plt.show()

In [ ]:
ctrl_pre = tgt_pre_grupo["EXP"]
ctrl_post = tgt_post_grupo["EXP"]

data_pre = [ctrl_pre[ctrl_pre['comp']==c].loc[:, ['Fz','Pz','Cz']].values.flatten() for c in components]
data_post = [ctrl_post[ctrl_post['comp']==c].loc[:, ['Fz','Pz','Cz']].values.flatten() for c in components]

import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

# Crear la figura con subgráficos
fig, axes = plt.subplots(1, len(components), figsize=(15, 5))

# Graficar cada par Pre/Post en un subplot
for i, c in enumerate(components):
    df = pd.DataFrame({
        "Valores": np.concatenate([data_pre[i], data_post[i]]),
        "Condición": ["Pre"] * len(data_pre[i]) + ["Post"] * len(data_post[i])
    })
    
    sns.boxplot(x="Condición", y="Valores", data=df, ax=axes[i])
    axes[i].set_title(f"Comparación para {c}")
    axes[i].set_xlabel("Condición")
    axes[i].set_ylabel("Valores")

# Ajustar el diseño y mostrar
plt.tight_layout()
plt.show()

In [ ]:
# dfs_t_tgt = {'PRE':[tgt_pre_grupo["CONTROL"], tgt_pre_grupo["PLCB"], tgt_pre_grupo["EXP"]],
#              'POST':[tgt_post_grupo["CONTROL"], tgt_post_grupo["PLCB"], tgt_post_grupo["EXP"]]}

# Time.plot_distributions(dfs_t_tgt, ['Pz','Cz'], components, title = "[Target] Tiempos de respuesta")

In [ ]:
# # Lista que guardara los resultados
# stats_results = []

# # Por cada componente, then:
# for comp in components:

#     # Definimos diccionario con todos los datos pareados a comparar
#     dict_time = {
#         'CTRL': [tgt_pre_grupo["CONTROL"][tgt_pre_grupo["CONTROL"]['comp']==comp].loc[:,channels], tgt_post_grupo["CONTROL"][tgt_post_grupo["CONTROL"]['comp']==comp].loc[:,channels]],
#         'PLCB': [tgt_pre_grupo["PLCB"][tgt_pre_grupo["PLCB"]['comp']==comp].loc[:,channels], tgt_post_grupo["PLCB"][tgt_post_grupo["PLCB"]['comp']==comp].loc[:,channels]],
#         'EXP': [tgt_pre_grupo["EXP"][tgt_pre_grupo["EXP"]['comp']==comp].loc[:,channels], tgt_post_grupo["EXP"][tgt_post_grupo["EXP"]['comp']==comp].loc[:,channels]]}

#     # Guardamos los resultados de los test pareados
#     stats_results.append(Entropy.compute_paired_test(dict_time, comp, "wilcoxon"))

# # Creamos un DataFrame con todos los estadisticos
# results_df = pd.concat(stats_results, ignore_index=True)

In [ ]:
# results_df[(results_df['grupo']=='CTRL')&(results_df['cohens_d']>0)&(results_df['adj_p_value']<0.05)]

In [ ]:
# results_df[(results_df['grupo']=='PLCB')&(results_df['cohens_d']>0)&(results_df['adj_p_value']<0.05)]

In [ ]:
# results_df[(results_df['grupo']=='EXP')&(results_df['cohens_d']>0)&(results_df['adj_p_value']<0.05)]

In [ ]:
# dfs_t_std = {'PRE':[df_t_std_pre_ctrl, df_t_std_pre_plcb, df_t_std_pre_exp],
#              'POST':[df_t_std_post_ctrl, df_t_std_post_plcb, df_t_std_post_exp]}


# Time.plot_distributions(dfs_t_std, cols, components, title = "[Standard] Tiempos de respuesta")

In [ ]:
# # Seleccionamos los canales sobre los que computar los estadisticos
# cols = np.array([channels.index(i) for i in best_channels], dtype=int)+6

# # Lista que guardara los resultados
# stats_results = []

# # Por cada componente, then:
# for comp in components:

#     # Definimos diccionario con todos los datos pareados a comparar
#     dict_time = {
#         'CTRL': [df_t_std_pre_ctrl[df_t_std_pre_ctrl['comp']==comp].iloc[:,cols], df_t_std_post_ctrl[df_t_std_post_ctrl['comp']==comp].iloc[:,cols]],
#         'PLCB': [df_t_std_pre_plcb[df_t_std_pre_plcb['comp']==comp].iloc[:,cols], df_t_std_post_plcb[df_t_std_post_plcb['comp']==comp].iloc[:,cols]],
#         'EXP': [df_t_std_pre_exp[df_t_std_pre_exp['comp']==comp].iloc[:,cols], df_t_std_post_exp[df_t_std_post_exp['comp']==comp].iloc[:,cols]]}

#     # Guardamos los resultados de los test pareados
#     stats_results.append(Entropy.compute_paired_test(dict_time, comp, "wilcoxon"))

# # Creamos un DataFrame con todos los estadisticos
# results_df = pd.concat(stats_results, ignore_index=True)

In [ ]:
# results_df[(results_df['grupo']=='CTRL')&(results_df['cohens_d']>0.2)&(results_df['adj_p_value']<0.05)]

In [ ]:
# results_df[(results_df['grupo']=='PLCB')&(results_df['cohens_d']>0.2)&(results_df['adj_p_value']<0.05)]

In [ ]:
# results_df[(results_df['grupo']=='EXP')&(results_df['cohens_d']>0.2)&(results_df['adj_p_value']<0.05)]

# Future tasks

- Trabajar a nivel de bandas cerebrales.
- Estudiar correlación cruzada, desfase y similitud entre señales. Consistencia en la periodicidad y diferencias en intensidad de deflexiones, entre otros.
- Estudiar correlaciones entre componentes ERP, es decir, ¿aumento de una componente implica la disminución de otra dependiendo de la intervención?.
- Estudiar la posibilidad del ajuste de un modelo por covariables y añadir la edad como covariable. 
- Analizar el tiempo de detección entre estímulos *target* y *standard*. Extrapolar conclusiones sobre el tiempo de respuesta dentro de cada grupo. 

Vamos a construir un modelo con la reserva cognitiva como variable respuesta y entropía, edad, media y varianza.

Luego añadimos construimos modelo de regresión logística

In [ ]:
eeg_groups = {
    "Frontal": ["Fp1", "Fp2", "F3", "Fz", "F4", "F7", "F8"],
    "Central": ["C3", "Cz", "C4"],
    "Parietal": ["P3", "Pz", "P4"],
    "Occipital": ["O1", "Oz", "O2"],
    "Temporal": ["T7", "T8"],
    "Parietotemporal": ["P7", "P8"],
    "Parieto-occipital": ["P3", "P4"]
}